# Part 1: Returns, Risk, and Factors – Replication

module import

In [45]:
import pandas as pd
import numpy as np
import os
import statsmodels.api as sm
from scipy.stats import f

according to teacher's email, we can use monthly data instead of daily data to do the regress in the sake of convenience.

In each datasheet, the meaning of conlumn names in the sheet can be found at the according txt file, for example, the meaning of "STK_MKT_FIVEFACMONTH.xlsx" can be found in "STK_MKT_FIVEFACMONTH\[DES\]\[xlsx\].txt".

In [46]:
#  check the current working directory
os.getcwd()

'/mnt/e/MyPythonProject/investment_project'

# 1.1 Import the factor_data

In [47]:
factor_data = pd.read_excel(
    'source_data/5-Factor Model Index (Monthly)/STK_MKT_FIVEFACMONTH.xlsx')  # warning means no error, let's just ignore and continue!

/home/msi/miniconda3/envs/rapids-24.04/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


clean the data: **factor_data**, during which we delete the unnecessary rows and columns, and rename the columns.

according to the definition of Portfolios, we only use the ones marked as 1.

note: Portfolios [Portfolio Type] - 1 represents 2\*3 portfolio division method; 2 represents 2\*2 portfolio division method; 3 represents 2\*2\*2\*2 portfolio division method.

In [48]:
factor_data = factor_data[
    ['MarkettypeID', 'TradingMonth', 'Portfolios', 'RiskPremium2', 'SMB2', 'HML2', 'RMW2', 'CMA2']]
factor_data.columns = ['MarkettypeID', 'date', 'portfolios', 'risk_premium', 'smb', 'hml', 'rmw', 'cma']
factor_data = factor_data.iloc[2:, :]

In [49]:
factor_data = factor_data[factor_data['portfolios'] == 1]
factor_data = factor_data[
    factor_data['MarkettypeID'] == "P9706"]  # P9706: SSE-SZSE A share market (excluding STAR Market, ChiNext)
factor_data.reset_index(drop=True, inplace=True)  # reset the index to 0,1,2,...

In [50]:
factor_data.head(5)  # check the first 5 rows

,MarkettypeID,date,portfolios,risk_premium,smb,hml,rmw,cma
0,P9706,2000-01,1,0.135225,-0.005175,-0.104151,0.042289,-0.076779
1,P9706,2000-02,1,0.113951,0.032327,-0.002393,-0.011365,0.03855
2,P9706,2000-03,1,0.058133,0.069624,0.01608,-0.051447,0.065904
3,P9706,2000-04,1,0.015578,-0.010779,0.02358,-0.022984,0.00972
4,P9706,2000-05,1,0.027197,0.025777,0.025355,-0.00662,-0.005409


In [51]:
factor_data.tail(5)  # check the last 5 rows

,MarkettypeID,date,portfolios,risk_premium,smb,hml,rmw,cma
284,P9706,2023-09,1,-0.002334,0.0091,0.015482,-0.00497,-0.003365
285,P9706,2023-10,1,-0.02938,0.026882,-0.009619,-0.016592,-0.003727
286,P9706,2023-11,1,-0.00157,0.052495,-0.008304,-0.021795,0.019977
287,P9706,2023-12,1,-0.015435,0.005892,0.002136,0.019856,-0.014981
288,P9706,2024-01,1,-0.057066,-0.108162,0.108981,-0.003561,-0.007933


# 1.2 Import the return data of each stock

In [52]:
stock_return = pd.read_excel(
    'source_data/Monthly Stock Price Returns/TRD_Mnth.xlsx')  # this is time-consuming, taking me about 41 seconds to load the data

/home/msi/miniconda3/envs/rapids-24.04/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [53]:
stock_return = stock_return[['Stkcd', 'Trdmnt',
                             'Mretnd']]  # calculate the return of each stock, which is the monthly return without cash dividend reinvested of each stock
stock_return = stock_return.iloc[2:, :]

In [54]:
stock_return.head(5)  # check the first 5 rows

,Stkcd,Trdmnt,Mretnd
2,000001,2000-01,0.061891
3,000001,2000-02,-0.011333
4,000001,2000-03,0.002729
5,000001,2000-04,0.037017
6,000001,2000-05,-0.055118


In [55]:
stock_return.shape  # check the shape of the data

(730499, 3)

# calculate the market risk premium

note that whatever method we use, the calculation of market risk premium is always the same

In [56]:
# import and clean the market return data
mkt_return = pd.read_excel('source_data/万得全A.xlsx')  # load the market return data
mkt_return = mkt_return.iloc[3:, :]
mkt_return.columns = ['date', 'mkt_index']
mkt_return['date'] = mkt_return['date'].apply(lambda x: x.strftime('%Y-%m-%d'))
mkt_return['date'] = pd.to_datetime(mkt_return['date'])
mkt_return.reset_index(drop=True, inplace=True)
mkt_return.head(5)

,date,mkt_index
0,1994-12-30,379.3402
1,1995-01-03,375.9952
2,1995-01-04,385.7705
3,1995-01-05,381.6151
4,1995-01-06,379.3633


In [57]:
mkt_return['daily_mkt_return'] = mkt_return['mkt_index'].pct_change()  # get the market index return
start_time = pd.to_datetime("2000-01-01")
start_time = np.array(start_time, dtype=np.datetime64)
end_time = np.array(pd.to_datetime("2024-01-01"), dtype=np.datetime64)
mkt_return = mkt_return[
    (mkt_return['date'] >= start_time) & (mkt_return['date'] < end_time)]  # get data in target time range
mkt_return.head(5)

/tmp/ipykernel_1676/1127946263.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mkt_return['daily_mkt_return'] = mkt_return['mkt_index'].pct_change()  # get the market index return


,date,mkt_index,daily_mkt_return
1227,2000-01-04,1031.0334,0.031033
1228,2000-01-05,1034.4385,0.003303
1229,2000-01-06,1076.512,0.040673
1230,2000-01-07,1117.7781,0.038333
1231,2000-01-10,1141.0309,0.020803


In [58]:
# import and clean the risk-free rate data
rf = pd.read_excel('source_data/Risk-Free Rate/TRD_Nrrate.xlsx')  # load the risk-free rate data
rf = rf.iloc[2:, :]
rf = rf[['Clsdt', 'Nrrdaydt','Nrrdata']]
rf.columns = ['date', 'daily_rf','rf']
rf['date'] = pd.to_datetime(rf['date'])
rf['daily_rf'] = rf['daily_rf'] / 100  # change the unit of the risk-free rate to decimal
rf['rf'] = rf['rf'] / 100  # change the unit of the risk-free rate to decimal
rf.head(5)

/home/msi/miniconda3/envs/rapids-24.04/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,date,daily_rf,rf
2,2000-01-01,0.000061,0.0225
3,2000-01-02,0.000061,0.0225
4,2000-01-03,0.000061,0.0225
5,2000-01-04,0.000061,0.0225
6,2000-01-05,0.000061,0.0225


In [59]:
rf['month'] = rf['date'].dt.to_period('M')
rf_monthly = rf.groupby('month').agg({'daily_rf': [lambda x: (1 + x).prod() - 1], 'rf': 'last'})
rf_monthly.reset_index(inplace=True)
rf_monthly.columns = ['month', 'monthly_rf','rf']
rf_monthly.head(5)

,month,monthly_rf,rf
0,2000-01,0.001893,0.0225
1,2000-02,0.001771,0.0225
2,2000-03,0.001893,0.0225
3,2000-04,0.001832,0.0225
4,2000-05,0.001893,0.0225


In [60]:
mkt_risk_premium = mkt_return.merge(rf, left_on='date', right_on='date',
                                    how='left')  # merge the market return data with the risk-free rate data
mkt_risk_premium['daily_mkt_risk_premium'] = mkt_risk_premium['daily_mkt_return'] - mkt_risk_premium[
    'daily_rf']  # calculate the market risk premium
mkt_risk_premium.head(5)  # so we get the market risk premium factor in target time range

,date,mkt_index,daily_mkt_return,daily_rf,rf,month,daily_mkt_risk_premium
0,2000-01-04,1031.0334,0.031033,0.000061,0.0225,2000-01,0.030972
1,2000-01-05,1034.4385,0.003303,0.000061,0.0225,2000-01,0.003242
2,2000-01-06,1076.512,0.040673,0.000061,0.0225,2000-01,0.040612
3,2000-01-07,1117.7781,0.038333,0.000061,0.0225,2000-01,0.038272
4,2000-01-10,1141.0309,0.020803,0.000061,0.0225,2000-01,0.020742


In [61]:
# for we will use monthly data to replicate Fama-French 5-factor model, we need to calculate the monthly market risk premium
mkt_risk_premium['year_month'] = mkt_risk_premium['date'].dt.to_period('M')
mkt_risk_premium = mkt_risk_premium.groupby('year_month').agg(
    {'daily_mkt_risk_premium': [lambda x: (1 + x).prod() - 1]})
mkt_risk_premium.reset_index(inplace=True)
mkt_risk_premium.columns = ['month', 'mkt_risk_premium']

In [62]:
mkt_risk_premium.head(5)

,month,mkt_risk_premium
0,2000-01,0.159452
1,2000-02,0.121265
2,2000-03,0.053877
3,2000-04,0.011737
4,2000-05,0.027313


# 1.3 solve the problem of point 3 in part 1

## 1.3.1 use variance to stand for the risk

the requirement in the pdf is "Report summary statistics of stock risk and returns“, so we just use the variance of the stock return as the risk of the stock.

In [63]:
stock_return_stat = stock_return.groupby('Stkcd').agg({'Mretnd': ['mean', 'var', 'min', 'max', 'std',
                                                                  lambda x: x.quantile(.25), lambda x: x.quantile(.5),
                                                                  lambda x: x.quantile(.75), 'skew', lambda
                                                                      x: x.kurt()]})  # calculate the mean and variance of the stock return
stock_return_stat.columns = ['mean', 'var', 'min', 'max', 'std', '25% quantile', '50% quantile', '75% quantile', 'skew',
                             'kurtosis']  # rename the columns

In [64]:
stock_return_stat.head(5)

,mean,var,min,max,std,25% quantile,50% quantile,75% quantile,skew,kurtosis
Stkcd,,,,,,,,,,
000001,0.009431,0.011582,-0.305195,0.37447,0.107620,-0.059481,0.004307,0.060023,0.47665,1.167765
000002,0.018041,0.01589,-0.296407,0.621102,0.126057,-0.056359,0.004945,0.078860,1.34035,4.507270
000003,-0.014095,0.028871,-0.475,0.281609,0.169914,-0.090354,-0.006667,0.089521,-0.728428,1.150957
000004,0.013136,0.022394,-0.389978,0.783523,0.149648,-0.079136,0.008670,0.096106,0.640154,2.904307
000005,0.010153,0.047893,-0.422857,2.858537,0.218845,-0.072539,-0.011299,0.053640,8.346999,105.749978


In [65]:
stock_return_stat.apply(
    'mean')  # calculate the mean of each column, which is the summary statistics of the whole market

mean            0.004765
var             0.028576
min            -0.333545
max             0.703519
std             0.153934
25% quantile   -0.078741
50% quantile   -0.008728
75% quantile    0.068607
skew            1.211271
kurtosis        7.506872
dtype: object

In [66]:
stock_return_stat.to_excel(r'output/part1/1.3 stock_return_stat.xlsx')  # save the result to an Excel file

variable "stock_return_stat" is the summary statistics of stock risk and returns: Stkcd means "stock code".

## 1.3.2 use the factor exposure to stand for the risk

if we use the according factor exposure to stand for the risk, denote $r_{it}$ as the return of stock i at time t, we should run the regression $$r_{it} = \alpha_i + \beta_{i1}f_{1t} + \beta_{i2}f_{2t} + \beta_{i3}f_{3t} + \beta_{i4}f_{4t} + \beta_{i5}f_{5t} + \epsilon_{it}, t=1,2,3,...,T$$ where $f_{1t}, f_{2t}, f_{3t}, f_{4t}, f_{5t}$ are the factors, $\beta_{i1}, \beta_{i2}, \beta_{i3}, \beta_{i4}, \beta_{i5}$ are the factor exposures, $\alpha_i$ is the alpha, and $\epsilon_{it}$ is the error term.

So we can get $\beta_{i1}, \beta_{i2}, \beta_{i3}, \beta_{i4}, \beta_{i5}$ as the risk of the stock. and finally we can have a table of the risk of each stock.

In [67]:
# regress each stock's return on the factors
tmp_return = stock_return.merge(factor_data, left_on='Trdmnt', right_on='date',
                                how='left')  # merge the stock return data with the factor data
tmp_return.head(5)

,Stkcd,Trdmnt,Mretnd,MarkettypeID,date,portfolios,risk_premium,smb,hml,rmw,cma
0,000001,2000-01,0.061891,P9706,2000-01,1,0.135225,-0.005175,-0.104151,0.042289,-0.076779
1,000001,2000-02,-0.011333,P9706,2000-02,1,0.113951,0.032327,-0.002393,-0.011365,0.03855
2,000001,2000-03,0.002729,P9706,2000-03,1,0.058133,0.069624,0.01608,-0.051447,0.065904
3,000001,2000-04,0.037017,P9706,2000-04,1,0.015578,-0.010779,0.02358,-0.022984,0.00972
4,000001,2000-05,-0.055118,P9706,2000-05,1,0.027197,0.025777,0.025355,-0.00662,-0.005409


In [68]:
tmp_return[tmp_return['Stkcd'] == '000001'].tail(5)

,Stkcd,Trdmnt,Mretnd,MarkettypeID,date,portfolios,risk_premium,smb,hml,rmw,cma
282,000001,2023-09,0.006289,P9706,2023-09,1,-0.002334,0.0091,0.015482,-0.00497,-0.003365
283,000001,2023-10,-0.066071,P9706,2023-10,1,-0.02938,0.026882,-0.009619,-0.016592,-0.003727
284,000001,2023-11,-0.07457,P9706,2023-11,1,-0.00157,0.052495,-0.008304,-0.021795,0.019977
285,000001,2023-12,-0.029959,P9706,2023-12,1,-0.015435,0.005892,0.002136,0.019856,-0.014981
286,000001,2024-01,0.007455,P9706,2024-01,1,-0.057066,-0.108162,0.108981,-0.003561,-0.007933


In [69]:
tmp_return[tmp_return['Stkcd'] == '000002'].tail(
    5)  # we can find that the result is the same as the one above, so our merge is correct

,Stkcd,Trdmnt,Mretnd,MarkettypeID,date,portfolios,risk_premium,smb,hml,rmw,cma
565,000002,2023-09,-0.041056,P9706,2023-09,1,-0.002334,0.0091,0.015482,-0.00497,-0.003365
566,000002,2023-10,-0.133792,P9706,2023-10,1,-0.02938,0.026882,-0.009619,-0.016592,-0.003727
567,000002,2023-11,0.009709,P9706,2023-11,1,-0.00157,0.052495,-0.008304,-0.021795,0.019977
568,000002,2023-12,-0.085664,P9706,2023-12,1,-0.015435,0.005892,0.002136,0.019856,-0.014981
569,000002,2024-01,-0.082218,P9706,2024-01,1,-0.057066,-0.108162,0.108981,-0.003561,-0.007933


In [70]:
# tmp_return.drop(columns=['date', 'rf', 'month'], inplace=True)
tmp_return.drop(columns=['MarkettypeID', 'date', 'portfolios'], inplace=True)
tmp_return.head(10)

,Stkcd,Trdmnt,Mretnd,risk_premium,smb,hml,rmw,cma
0,000001,2000-01,0.061891,0.135225,-0.005175,-0.104151,0.042289,-0.076779
1,000001,2000-02,-0.011333,0.113951,0.032327,-0.002393,-0.011365,0.03855
2,000001,2000-03,0.002729,0.058133,0.069624,0.01608,-0.051447,0.065904
3,000001,2000-04,0.037017,0.015578,-0.010779,0.02358,-0.022984,0.00972
4,000001,2000-05,-0.055118,0.027197,0.025777,0.025355,-0.00662,-0.005409
5,000001,2000-06,0.007222,0.022145,-0.026737,0.032628,0.002911,0.002796
6,000001,2000-07,0.02096,0.044277,0.017068,0.03052,-0.002764,0.045901
7,000001,2000-08,-0.041059,-0.008596,0.042098,-0.017277,-0.032858,0.013269
8,000001,2000-09,-0.044507,-0.049773,0.028192,-0.034215,-0.003422,0.011796
9,000001,2000-10,0.034788,0.023503,0.032889,0.013666,0.001559,0.00692


In [71]:
# tmp_return['Trdmnt'] = (tmp_return['Trdmnt']).astype(str)
tmp_return['Trdmnt'] = pd.to_datetime(tmp_return['Trdmnt']).dt.to_period('M')
tmp_return.shape

(730499, 8)

In [72]:
# add rf to tmp_return, this is a little time-consuming (1 min)
tmp_return['daily_rf'] = tmp_return['Trdmnt'].apply(lambda x: rf[rf['month'] == x]['daily_rf'].values[0])
tmp_return.shape

(730499, 9)

In [73]:
tmp_return['excess_return'] = tmp_return['Mretnd'] - tmp_return['daily_rf']  # calculate the excess return
tmp_return.head(10)

,Stkcd,Trdmnt,Mretnd,risk_premium,smb,hml,rmw,cma,daily_rf,excess_return
0,000001,2000-01,0.061891,0.135225,-0.005175,-0.104151,0.042289,-0.076779,0.000061,0.06183
1,000001,2000-02,-0.011333,0.113951,0.032327,-0.002393,-0.011365,0.03855,0.000061,-0.011394
2,000001,2000-03,0.002729,0.058133,0.069624,0.01608,-0.051447,0.065904,0.000061,0.002668
3,000001,2000-04,0.037017,0.015578,-0.010779,0.02358,-0.022984,0.00972,0.000061,0.036956
4,000001,2000-05,-0.055118,0.027197,0.025777,0.025355,-0.00662,-0.005409,0.000061,-0.055179
5,000001,2000-06,0.007222,0.022145,-0.026737,0.032628,0.002911,0.002796,0.000061,0.007161
6,000001,2000-07,0.02096,0.044277,0.017068,0.03052,-0.002764,0.045901,0.000061,0.020899
7,000001,2000-08,-0.041059,-0.008596,0.042098,-0.017277,-0.032858,0.013269,0.000061,-0.04112
8,000001,2000-09,-0.044507,-0.049773,0.028192,-0.034215,-0.003422,0.011796,0.000061,-0.044568
9,000001,2000-10,0.034788,0.023503,0.032889,0.013666,0.001559,0.00692,0.000061,0.034727


In [74]:
tmp_return.head(5)

,Stkcd,Trdmnt,Mretnd,risk_premium,smb,hml,rmw,cma,daily_rf,excess_return
0,000001,2000-01,0.061891,0.135225,-0.005175,-0.104151,0.042289,-0.076779,0.000061,0.06183
1,000001,2000-02,-0.011333,0.113951,0.032327,-0.002393,-0.011365,0.03855,0.000061,-0.011394
2,000001,2000-03,0.002729,0.058133,0.069624,0.01608,-0.051447,0.065904,0.000061,0.002668
3,000001,2000-04,0.037017,0.015578,-0.010779,0.02358,-0.022984,0.00972,0.000061,0.036956
4,000001,2000-05,-0.055118,0.027197,0.025777,0.025355,-0.00662,-0.005409,0.000061,-0.055179


In [75]:
# this cell costs me 6 seconds to run
def regress(data):
    X = data[['risk_premium', 'smb', 'hml', 'rmw', 'cma']]
    X = sm.add_constant(X)
    y = data['excess_return']
    model = sm.OLS(np.array(y.astype(float)), X.astype(float))
    result = model.fit()
    params_and_tvalues = np.append(result.params, result.tvalues)
    return params_and_tvalues


betas = (tmp_return.groupby('Stkcd').apply(regress)).apply(
    pd.Series)  # calculate the correlation between the stock return and the risk premium
betas.columns = ['const', 'risk_premium', 'smb', 'hml', 'rmw', 'cma', 't-const', 't-risk_premium', 't-smb', 't-hml',
                 't-rmw', 't-cma']

/tmp/ipykernel_1676/3057413395.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  betas = (tmp_return.groupby('Stkcd').apply(regress)).apply(


In [76]:
betas.head(5)

,const,risk_premium,smb,hml,rmw,cma,t-const,t-risk_premium,t-smb,t-hml,t-rmw,t-cma
Stkcd,,,,,,,,,,,,
000001,0.007879,1.102400,-0.682552,0.190543,-0.575781,-0.319408,1.885009,18.450228,-4.308816,1.154474,-2.193837,-1.161902
000002,0.013725,1.072140,-0.190162,0.200709,0.556242,-0.416970,2.208393,11.853675,-0.805694,0.830612,1.439427,-1.035893
000003,-0.061240,2.074528,-0.105670,-0.061860,-0.572768,1.563042,-1.799507,3.720932,-0.073680,-0.051804,-0.284718,0.829056
000004,-0.000726,0.921272,1.471728,-0.034041,-0.498565,-0.777319,-0.108083,9.689016,5.818053,-0.130163,-1.199613,-1.792275
000005,0.000283,1.137911,0.925412,-1.537135,-0.325210,0.851721,0.025462,7.251842,2.213628,-3.345554,-0.473892,1.163733


In [77]:
betas.to_excel(r'output/part1/1.3 betas.xlsx')  # save the result to an Excel file

variable "betas" is the risk of each stock: Stkcd means "stock code", const means the alpha, risk_premium means the risk premium, smb means the size factor, hml means the value factor, rmw means the profitability factor, cma means the investment factor.

we will do a mix-OLS regression to get the factor exposure.

In [78]:
X = tmp_return.dropna()[['risk_premium', 'smb', 'hml', 'rmw', 'cma']]
X = sm.add_constant(X)
Y = tmp_return.dropna()['excess_return']
model = sm.OLS(Y.astype(float), X.astype(float))
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          excess_return   R-squared:                       0.264
Model:                            OLS   Adj. R-squared:                  0.264
Method:                 Least Squares   F-statistic:                 5.210e+04
Date:                Sat, 22 Jun 2024   Prob (F-statistic):               0.00
Time:                        10:51:52   Log-Likelihood:             4.0163e+05
No. Observations:              725828   AIC:                        -8.032e+05
Df Residuals:                  725822   BIC:                        -8.032e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.0027      0.000     16.222      0.000       0.002       0.003
risk_premium     1.0184      0.003    382.195      0.000       1.013       1.024
smb              0.9000      0.006    143.434      0.000       0.888       0.912
hml             -0.1168      0.007    -16.926      0.000      -0.130      -0.103
rmw             -0.0255      0.010     -2.435      0.015      -0.046      -0.005
cma              0.0449      0.011      3.911      0.000       0.022       0.067
==============================================================================
Omnibus:                  1529892.963   Durbin-Watson:                   2.067
Prob(Omnibus):                  0.000   Jarque-Bera (JB):      68285581118.121
Skew:                          17.359   Prob(JB):                         0.00
Kurtosis:                    1505.233   Cond. No.                         86.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [79]:
with open(r'output/part1/1.3 mix_ols_summary1.html', 'w') as tar_file:
    tar_file.write(result.summary().as_html())
with open(r'output/part1/1.3 mix_ols_summary2.html', 'w') as tar_file:
    tar_file.write(result.summary2().as_html())

# 1.4 replicate Fama-French 5-factor model

There're altogather 3 methods in the paper to construct the 5-factor model, we will construct sequentially.

No matter what method we use, the formula is always the same, 

$$R_{it}-R_{Ft}=a_i+b_i(R_{Mt}-R_{Ft})+s_iSMB_t+h_i HML_t+r_iRMW_t+c_iCMA_t+e_{it}$$

where $R_{it}$ is the return of stock i at time t, $R_{Ft}$ is the risk-free rate at time t, $R_{Mt}$ is the market return at time t, $SMB_t$ is the size factor at time t, $HML_t$ is the value factor at time t, $RMW_t$ is the profitability factor at time t, $CMA_t$ is the investment factor at time t, $a_i$ is the alpha of stock i, $b_i$ is the beta of the market return, $s_i$ is the beta of the size factor, $h_i$ is the beta of the value factor, $r_i$ is the beta of the profitability factor, $c_i$ is the beta of the investment factor, $e_{it}$ is the error term.

the construction process of the 5-factor model is as follows:(from fama-french 5-factor model.pdf, see it in [here](papers/Fama-French A five-factor asset pricing model.pdf)

![ff5.png](img/ff5.png)

Similarly, we need to calculate the monthly stock price returns from daily data.

In [80]:
"""
import daily stock price return data, and calculate the monthly stock price returns
because this step is really, really time-consuming, (about 13 min on i7-13700K with PCIE 4.0 SSD)
there's altogether over 14 million rows in the daily stock price return data, 
we only run this cell once, and then save the result to an Excel file in ./output/monthly_stock_return.xlsx 
"""
# daily_stock_return = pd.read_excel('source_data/Daily Stock Price Returns 2000-2004/TRD_Dalyr.xlsx')  # load the daily stock price return data
# daily_stock_return = daily_stock_return.iloc[2:,:]
# tmp1 = pd.read_excel('source_data/Daily Stock Price Returns 2000-2004/TRD_Dalyr1.xlsx')  
# tmp1 = tmp1.iloc[2:,:]
# tmp2 = pd.read_excel('source_data/Daily Stock Price Returns 2005-2009/TRD_Dalyr.xlsx')
# tmp2 = tmp2.iloc[2:,:]
# tmp3 = pd.read_excel('source_data/Daily Stock Price Returns 2005-2009/TRD_Dalyr1.xlsx')
# tmp3 = tmp3.iloc[2:,:]
# tmp4 = pd.read_excel('source_data/Daily Stock Price Returns 2010-2014/TRD_Dalyr.xlsx')
# tmp4 = tmp4.iloc[2:,:]
# tmp5 = pd.read_excel('source_data/Daily Stock Price Returns 2010-2014/TRD_Dalyr1.xlsx')
# tmp5 = tmp5.iloc[2:,:]
# tmp6 = pd.read_excel('source_data/Daily Stock Price Returns 2010-2014/TRD_Dalyr2.xlsx')
# tmp6 = tmp6.iloc[2:,:]
# tmp7 = pd.read_excel('source_data/Daily Stock Price Returns 2015-2019/TRD_Dalyr.xlsx')
# tmp7 = tmp7.iloc[2:,:]
# tmp8 = pd.read_excel('source_data/Daily Stock Price Returns 2015-2019/TRD_Dalyr1.xlsx')
# tmp8 = tmp8.iloc[2:,:]
# tmp9 = pd.read_excel('source_data/Daily Stock Price Returns 2015-2019/TRD_Dalyr2.xlsx')
# tmp9 = tmp9.iloc[2:,:]
# tmp10 = pd.read_excel('source_data/Daily Stock Price Returns 2015-2019/TRD_Dalyr3.xlsx')
# tmp10 = tmp10.iloc[2:,:]
# tmp11 = pd.read_excel('source_data/Daily Stock Price Returns 2020-2024/TRD_Dalyr.xlsx')
# tmp11 = tmp11.iloc[2:,:]
# tmp12 = pd.read_excel('source_data/Daily Stock Price Returns 2020-2024/TRD_Dalyr1.xlsx')
# tmp12 = tmp12.iloc[2:,:]
# tmp13 = pd.read_excel('source_data/Daily Stock Price Returns 2020-2024/TRD_Dalyr2.xlsx')
# tmp13 = tmp13.iloc[2:,:]
# tmp14 = pd.read_excel('source_data/Daily Stock Price Returns 2020-2024/TRD_Dalyr3.xlsx')
# tmp14 = tmp14.iloc[2:,:]
# tmp15 = pd.read_excel('source_data/Daily Stock Price Returns 2020-2024/TRD_Dalyr4.xlsx')
# tmp15 = tmp15.iloc[2:,:]
# 
# daily_stock_return = pd.concat([daily_stock_return, tmp1, tmp2, tmp3, tmp4, tmp5, tmp6, tmp7, tmp8, tmp9, tmp10, tmp11, tmp12, tmp13, tmp14, tmp15], axis=0)  # concatenate the two dataframes
# 
# daily_stock_return.reset_index(drop=True, inplace=True)
# daily_stock_return.to_feather(r'temp/daily_stock_return.feather')  # save the result to a feather file, which is much more quickly than csv or xlsx
# 
# del tmp1, tmp2, tmp3, tmp4, tmp5, tmp6, tmp7, tmp8, tmp9, tmp10, tmp11, tmp12, tmp13, tmp14, tmp15  # delete the temporary dataframes

"\nimport daily stock price return data, and calculate the monthly stock price returns\nbecause this step is really, really time-consuming, (about 13 min on i7-13700K with PCIE 4.0 SSD)\nthere's altogether over 14 million rows in the daily stock price return data, \nwe only run this cell once, and then save the result to an Excel file in ./output/monthly_stock_return.xlsx \n"

In [81]:
daily_stock_return = pd.read_feather(r'temp/daily_stock_return.feather')  # load the monthly stock price return data

In [82]:
daily_stock_return = daily_stock_return[['Stkcd', 'Trddt', 'Dsmvtll', 'Dretwd']]
daily_stock_return.columns = ['Stkcd', 'date', 'market_value', 'daily_stock_return']
daily_stock_return['date'] = pd.to_datetime(daily_stock_return['date'])

In [83]:
daily_stock_return.head(20)

,Stkcd,date,market_value,daily_stock_return
0,000001,2000-01-04,28383283.31,0.048138
1,000001,2000-01-05,28026358.48,-0.012575
2,000001,2000-01-06,29143688.39,0.039867
3,000001,2000-01-07,30323092.18,0.040469
4,000001,2000-01-10,31254200.43,0.030706
5,000001,2000-01-11,29485094.75,-0.056604
6,000001,2000-01-12,28305690.96,-0.040000
7,000001,2000-01-13,28150506.25,-0.005482
8,000001,2000-01-14,27747026.00,-0.014333
9,000001,2000-01-17,28057395.42,0.011186


## 1.4.1 Method 1: Construct the 5-factor model using 2$\times$3 portfolio division method

In [84]:
daily_stock_return['month'] = daily_stock_return['date'].dt.to_period('M')
daily_stock_return = daily_stock_return[['Stkcd', 'date', 'month', 'daily_stock_return', 'market_value']]

In [85]:
# calculate monthly return for each stock
"""
this cell is a little time-consuming, taking me about 30 seconds to run with i7-13700K, so I just run it once and save the result to an Excel file in ./temp/monthly_stock_return.feather
"""
# monthly_stock_return = daily_stock_return.groupby(['Stkcd', 'month']).agg({'daily_stock_return': lambda x: (1+x).prod()-1, 'market_value':'last'})  # calculate the monthly return for each stock
# monthly_stock_return.columns = ['monthly_stock_return', 'market_value']
# monthly_stock_return.reset_index(inplace=True)
# monthly_stock_return.to_feather(r'temp/monthly_stock_return.feather')  # save the result to a feather file

'\nthis cell is a little time-consuming, taking me about 30 seconds to run with i7-13700K, so I just run it once and save the result to an Excel file in ./temp/monthly_stock_return.feather\n'

In [86]:
monthly_stock_return = pd.read_feather(r'temp/monthly_stock_return.feather')  # load the monthly stock price return data

### now we will give a tag on S on B, according to the market value of the stock.

In [87]:
def small_or_big(x):
    return pd.qcut(x, 2, labels=['S', 'B'])


monthly_stock_return['S or B'] = monthly_stock_return.groupby("month")['market_value'].transform(
    small_or_big)  # divide the stocks into 2 groups according to the market value
monthly_stock_return['market_value'] = monthly_stock_return[
                                           'market_value'] * 1000  # change the unit of the market value to yuan
monthly_stock_return.head(10)

,Stkcd,month,market_value,monthly_stock_return,S or B
2,000001,2000-01,2.875573e+10,0.061891,B
3,000001,2000-02,2.842984e+10,-0.011333,B
4,000001,2000-03,2.850743e+10,0.002729,B
5,000001,2000-04,2.956269e+10,0.037017,B
6,000001,2000-05,2.793325e+10,-0.055118,B
7,000001,2000-06,2.813499e+10,0.007222,B
8,000001,2000-07,2.872469e+10,0.020960,B
9,000001,2000-08,2.754529e+10,-0.041059,B
10,000001,2000-09,2.631933e+10,-0.044507,B
11,000001,2000-10,2.723492e+10,0.034788,B


### now we will give a tag on H, N or L, according to the book-to-market ratio of the stock.

In [88]:
# balance_sheet = pd.read_excel('source_data/Balance Sheet/FS_Combas.xlsx', skiprows=1)  # load the balance sheet data, using 28s
# balance_sheet.head(5)
# balance_sheet.to_csv(r'source_data/Balance Sheet/balance_sheet.csv')  # save the result to a csv file for quicker loading

In [89]:
balance_sheet = pd.read_csv(r'source_data/Balance Sheet/balance_sheet.csv')  # load the balance sheet data
balance_sheet = balance_sheet[balance_sheet['Statement Type'] == "A"]
balance_sheet.head(5)

/tmp/ipykernel_1676/2438934347.py:1: DtypeWarning: Columns (1,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  balance_sheet = pd.read_csv(r'source_data/Balance Sheet/balance_sheet.csv')  # load the balance sheet data


,Unnamed: 0,Stock Code,Stock Short Name,Ending Date of Statistics,Statement Type,Total Current Assets,Total Assets,Total Shareholders’ Equity
1,1,000001,SFZA,2000-01-01,A,NaN,43912394151,1141603885
2,2,000001,SFZA,2000-06-30,A,NaN,49732336516,3078512556
3,3,000001,SFZA,2000-12-31,A,NaN,67227499769,4738883655
4,4,000001,SFZA,2001-01-01,A,NaN,66006167607,3517551493
5,5,000001,SFZA,2001-06-30,A,NaN,85181426762,4961824149


In [90]:
balance_sheet = balance_sheet[['Stock Code', 'Ending Date of Statistics', 'Total Assets', 'Total Shareholders’ Equity']]
balance_sheet.columns = ['Stkcd', 'date', 'total_assets', 'total_shareholders_equity']
balance_sheet = balance_sheet.iloc[1:, :]
balance_sheet.head(5)

,Stkcd,date,total_assets,total_shareholders_equity
2,000001,2000-06-30,49732336516,3078512556
3,000001,2000-12-31,67227499769,4738883655
4,000001,2001-01-01,66006167607,3517551493
5,000001,2001-06-30,85181426762,4961824149
6,000001,2001-12-31,120126983351,3627668792


In [91]:
balance_sheet['date'] = pd.to_datetime(balance_sheet['date'])
balance_sheet['month'] = balance_sheet['date'].dt.to_period('M')
balance_sheet = balance_sheet[['Stkcd', 'month', 'total_assets', 'total_shareholders_equity']]
balance_sheet.head(5)

,Stkcd,month,total_assets,total_shareholders_equity
2,000001,2000-06,49732336516,3078512556
3,000001,2000-12,67227499769,4738883655
4,000001,2001-01,66006167607,3517551493
5,000001,2001-06,85181426762,4961824149
6,000001,2001-12,120126983351,3627668792


In [92]:
monthly_stock_return = monthly_stock_return.merge(balance_sheet, left_on=['Stkcd', 'month'],
                                                  right_on=['Stkcd', 'month'], how='left')
monthly_stock_return['total_assets'] = monthly_stock_return['total_assets'].ffill().astype(
    np.float64)  # ffill means use the value of the last row to fill the missing value
monthly_stock_return['total_shareholders_equity'] = monthly_stock_return['total_shareholders_equity'].ffill().astype(
    np.float64)
monthly_stock_return.head(10)

,Stkcd,month,market_value,monthly_stock_return,S or B,total_assets,total_shareholders_equity
0,000001,2000-01,2.875573e+10,0.061891,B,NaN,NaN
1,000001,2000-02,2.842984e+10,-0.011333,B,NaN,NaN
2,000001,2000-03,2.850743e+10,0.002729,B,NaN,NaN
3,000001,2000-04,2.956269e+10,0.037017,B,NaN,NaN
4,000001,2000-05,2.793325e+10,-0.055118,B,NaN,NaN
5,000001,2000-06,2.813499e+10,0.007222,B,4.973234e+10,3.078513e+09
6,000001,2000-07,2.872469e+10,0.020960,B,4.973234e+10,3.078513e+09
7,000001,2000-08,2.754529e+10,-0.041059,B,4.973234e+10,3.078513e+09
8,000001,2000-09,2.631933e+10,-0.044507,B,4.973234e+10,3.078513e+09
9,000001,2000-10,2.723492e+10,0.034788,B,4.973234e+10,3.078513e+09


In [93]:
monthly_stock_return['BM ratio'] = monthly_stock_return['total_shareholders_equity'] / monthly_stock_return[
    'market_value']  # calculate the book-to-market ratio
monthly_stock_return.head(10)

,Stkcd,month,market_value,monthly_stock_return,S or B,total_assets,total_shareholders_equity,BM ratio
0,000001,2000-01,2.875573e+10,0.061891,B,NaN,NaN,NaN
1,000001,2000-02,2.842984e+10,-0.011333,B,NaN,NaN,NaN
2,000001,2000-03,2.850743e+10,0.002729,B,NaN,NaN,NaN
3,000001,2000-04,2.956269e+10,0.037017,B,NaN,NaN,NaN
4,000001,2000-05,2.793325e+10,-0.055118,B,NaN,NaN,NaN
5,000001,2000-06,2.813499e+10,0.007222,B,4.973234e+10,3.078513e+09,0.109419
6,000001,2000-07,2.872469e+10,0.020960,B,4.973234e+10,3.078513e+09,0.107173
7,000001,2000-08,2.754529e+10,-0.041059,B,4.973234e+10,3.078513e+09,0.111762
8,000001,2000-09,2.631933e+10,-0.044507,B,4.973234e+10,3.078513e+09,0.116968
9,000001,2000-10,2.723492e+10,0.034788,B,4.973234e+10,3.078513e+09,0.113036


In [94]:
def H_N_or_L(x):
    return pd.qcut(x, q=[0, .3, .7, 1], labels=['H', 'N', 'L'])


monthly_stock_return['H, N or L'] = monthly_stock_return.groupby("month")['BM ratio'].transform(
    H_N_or_L)  # divide the stocks into 3 groups according to the book-to-market ratio

In [95]:
monthly_stock_return.head(10)

,Stkcd,month,market_value,monthly_stock_return,S or B,total_assets,total_shareholders_equity,BM ratio,"H, N or L"
0,000001,2000-01,2.875573e+10,0.061891,B,NaN,NaN,NaN,NaN
1,000001,2000-02,2.842984e+10,-0.011333,B,NaN,NaN,NaN,NaN
2,000001,2000-03,2.850743e+10,0.002729,B,NaN,NaN,NaN,NaN
3,000001,2000-04,2.956269e+10,0.037017,B,NaN,NaN,NaN,NaN
4,000001,2000-05,2.793325e+10,-0.055118,B,NaN,NaN,NaN,NaN
5,000001,2000-06,2.813499e+10,0.007222,B,4.973234e+10,3.078513e+09,0.109419,H
6,000001,2000-07,2.872469e+10,0.020960,B,4.973234e+10,3.078513e+09,0.107173,H
7,000001,2000-08,2.754529e+10,-0.041059,B,4.973234e+10,3.078513e+09,0.111762,H
8,000001,2000-09,2.631933e+10,-0.044507,B,4.973234e+10,3.078513e+09,0.116968,H
9,000001,2000-10,2.723492e+10,0.034788,B,4.973234e+10,3.078513e+09,0.113036,H


### now we can tag and calculate $SMB_{B/M}$

In [96]:
monthly_stock_return['SMB_{B/M}'] = monthly_stock_return['S or B'].astype(str) + monthly_stock_return[
    'H, N or L'].astype(str)  # combine the two tags
monthly_stock_return['SMB_{B/M}'] = monthly_stock_return['SMB_{B/M}'].astype('category')
monthly_stock_return = monthly_stock_return[
    ['Stkcd', 'month', 'monthly_stock_return', 'market_value', 'total_assets', 'total_shareholders_equity', 'BM ratio',
     'S or B', 'H, N or L', 'SMB_{B/M}']]
monthly_stock_return.head(10)

,Stkcd,month,monthly_stock_return,market_value,total_assets,total_shareholders_equity,BM ratio,S or B,"H, N or L",SMB_{B/M}
0,000001,2000-01,0.061891,2.875573e+10,NaN,NaN,NaN,B,NaN,Bnan
1,000001,2000-02,-0.011333,2.842984e+10,NaN,NaN,NaN,B,NaN,Bnan
2,000001,2000-03,0.002729,2.850743e+10,NaN,NaN,NaN,B,NaN,Bnan
3,000001,2000-04,0.037017,2.956269e+10,NaN,NaN,NaN,B,NaN,Bnan
4,000001,2000-05,-0.055118,2.793325e+10,NaN,NaN,NaN,B,NaN,Bnan
5,000001,2000-06,0.007222,2.813499e+10,4.973234e+10,3.078513e+09,0.109419,B,H,BH
6,000001,2000-07,0.020960,2.872469e+10,4.973234e+10,3.078513e+09,0.107173,B,H,BH
7,000001,2000-08,-0.041059,2.754529e+10,4.973234e+10,3.078513e+09,0.111762,B,H,BH
8,000001,2000-09,-0.044507,2.631933e+10,4.973234e+10,3.078513e+09,0.116968,B,H,BH
9,000001,2000-10,0.034788,2.723492e+10,4.973234e+10,3.078513e+09,0.113036,B,H,BH


In [97]:
# get weighted retrun of each SMB_{B/M} group
source_group = monthly_stock_return.groupby(['month', 'SMB_{B/M}'], observed=False).apply(
    lambda x: np.average(x['monthly_stock_return'], weights=x['market_value']), include_groups=False).unstack()
source_group.fillna(0, inplace=True)
source_group.reset_index(inplace=True)
source_group.set_index('month', inplace=True)
source_group = source_group.rename_axis('SMB_{B/M}').rename_axis('month', axis=1)
source_group['SMB_{B/M}'] = (source_group['SH'] + source_group['SN'] + source_group['SL']) / 3 - (
        source_group['BH'] + source_group['BN'] + source_group['BL']) / 3
source_group.head(10)

month,BH,BL,BN,Bnan,SH,SL,SN,SMB_{B/M}
SMB_{B/M},,,,,,,,
2000-01,0.211263,0.000000,0.0,0.061891,0.115403,0.000000,0.096189,0.000110
2000-02,0.199451,0.000000,0.0,-0.011333,0.124140,0.000000,0.096888,0.007192
2000-03,0.083746,0.000000,0.0,0.002729,0.127617,0.125206,0.108970,0.092683
2000-04,0.055933,0.000000,0.0,0.037017,0.001406,0.000000,0.026939,-0.009196
2000-05,0.032373,0.000000,0.0,-0.055118,0.038882,0.053875,0.000000,0.020128
2000-06,0.064659,0.000000,0.0,0.000000,0.015243,0.000000,0.000000,-0.016472
2000-07,0.046614,0.000000,0.0,0.000000,0.060891,0.000000,0.000000,0.004759
2000-08,-0.009001,0.000000,0.0,0.000000,0.013230,0.000000,0.018193,0.013475
2000-09,0.000000,-0.037499,0.0,0.000000,0.000000,0.000000,-0.033494,0.001335


### now we can tag and calculate $SMB_{OP}$

OP: operating profitabily, which can be measured by ROE, so we need net income and stockholders' equity to calculate ROE.
reference: [ff5](https://bigquant.com/wiki/doc/yinzi-moxing-hiQfGBEwbG)

In [98]:
income_statement = pd.read_excel(
    'source_data/Income Statement/FS_Comins.xlsx')  # load the income statement data, a little time-consuming, 23s
income_statement = income_statement[income_statement['Typrep'] == 'A']
income_statement = income_statement.iloc[2:, :]
income_statement = income_statement[['Stkcd', 'Accper', 'B002000000']]
income_statement.columns = ['Stkcd', 'date', 'net_income']
income_statement['date'] = pd.to_datetime(income_statement['date'])
income_statement['month'] = income_statement['date'].dt.to_period('M')
income_statement = income_statement[['Stkcd', 'month', 'net_income']]
income_statement.head(5)

/home/msi/miniconda3/envs/rapids-24.04/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Stkcd,month,net_income
4,000001,2000-12,506551785
5,000001,2001-01,462975563
6,000001,2001-06,223211685
7,000001,2001-12,402360428
8,000001,2002-01,402360428


In [99]:
balance_sheet.head(5)

,Stkcd,month,total_assets,total_shareholders_equity
2,000001,2000-06,49732336516,3078512556
3,000001,2000-12,67227499769,4738883655
4,000001,2001-01,66006167607,3517551493
5,000001,2001-06,85181426762,4961824149
6,000001,2001-12,120126983351,3627668792


In [100]:
# merge balance sheet and income statement data
balance_sheet = balance_sheet.merge(income_statement, on=['Stkcd', 'month'], how='left')
balance_sheet['net_income'] = balance_sheet['net_income'].ffill().astype(np.float64)
balance_sheet['total_shareholders_equity'] = balance_sheet['total_shareholders_equity'].astype(np.float64)
balance_sheet['ROE'] = balance_sheet['net_income'] / balance_sheet['total_shareholders_equity']
balance_sheet['ROE'] = balance_sheet['ROE'].replace([np.inf, -np.inf], np.nan)
balance_sheet['ROE'] = balance_sheet['ROE'].fillna(0)
balance_sheet.head(5)

/tmp/ipykernel_1676/2918935053.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  balance_sheet['net_income'] = balance_sheet['net_income'].ffill().astype(np.float64)


,Stkcd,month,total_assets,total_shareholders_equity,net_income,ROE
0,000001,2000-06,49732336516,3.078513e+09,NaN,0.000000
1,000001,2000-12,67227499769,4.738884e+09,506551785.0,0.106893
2,000001,2001-01,66006167607,3.517551e+09,462975563.0,0.131619
3,000001,2001-06,85181426762,4.961824e+09,223211685.0,0.044986
4,000001,2001-12,120126983351,3.627669e+09,402360428.0,0.110914


In [101]:
def R_N_or_W(x):
    edge1 = x.quantile(.3)
    edge2 = x.quantile(.7)
    tmp = x.apply(lambda y: 'R' if y > edge2 else 'W' if y < edge1 else 'N')
    return tmp


balance_sheet['R, N or W'] = balance_sheet.groupby("month")['ROE'].transform(
    R_N_or_W)  # divide the stocks into 3 groups according to the ROE
balance_sheet['R, N or W'] = balance_sheet['R, N or W'].astype('category')
balance_sheet.head(10)

,Stkcd,month,total_assets,total_shareholders_equity,net_income,ROE,"R, N or W"
0,000001,2000-06,49732336516,3.078513e+09,NaN,0.000000,W
1,000001,2000-12,67227499769,4.738884e+09,506551785.0,0.106893,W
2,000001,2001-01,66006167607,3.517551e+09,462975563.0,0.131619,W
3,000001,2001-06,85181426762,4.961824e+09,223211685.0,0.044986,W
4,000001,2001-12,120126983351,3.627669e+09,402360428.0,0.110914,W
5,000001,2002-01,120126983351,3.627669e+09,402360428.0,0.110914,W
6,000001,2002-03,110688552564,3.811916e+09,183951144.0,0.048257,W
7,000001,2002-06,134689020270,3.941856e+09,314141342.0,0.079694,W
8,000001,2002-09,160021970844,4.058194e+09,419642032.0,0.103406,W
9,000001,2002-12,166166379400,3.768021e+09,432224930.0,0.114709,W


In [102]:
# monthly_stock_return.drop(columns=['ROE_x','ROE_y','BM ratio', 'H, N or L'], inplace=True)
monthly_stock_return.head(10)

,Stkcd,month,monthly_stock_return,market_value,total_assets,total_shareholders_equity,BM ratio,S or B,"H, N or L",SMB_{B/M}
0,000001,2000-01,0.061891,2.875573e+10,NaN,NaN,NaN,B,NaN,Bnan
1,000001,2000-02,-0.011333,2.842984e+10,NaN,NaN,NaN,B,NaN,Bnan
2,000001,2000-03,0.002729,2.850743e+10,NaN,NaN,NaN,B,NaN,Bnan
3,000001,2000-04,0.037017,2.956269e+10,NaN,NaN,NaN,B,NaN,Bnan
4,000001,2000-05,-0.055118,2.793325e+10,NaN,NaN,NaN,B,NaN,Bnan
5,000001,2000-06,0.007222,2.813499e+10,4.973234e+10,3.078513e+09,0.109419,B,H,BH
6,000001,2000-07,0.020960,2.872469e+10,4.973234e+10,3.078513e+09,0.107173,B,H,BH
7,000001,2000-08,-0.041059,2.754529e+10,4.973234e+10,3.078513e+09,0.111762,B,H,BH
8,000001,2000-09,-0.044507,2.631933e+10,4.973234e+10,3.078513e+09,0.116968,B,H,BH
9,000001,2000-10,0.034788,2.723492e+10,4.973234e+10,3.078513e+09,0.113036,B,H,BH


In [103]:
# merge ROE information into monthly_stock_return
monthly_stock_return = monthly_stock_return.merge(balance_sheet[['Stkcd', 'month', 'ROE', 'R, N or W']],
                                                  left_on=['Stkcd', 'month'], right_on=['Stkcd', 'month'], how='left')
monthly_stock_return['ROE'] = monthly_stock_return['ROE'].ffill()
monthly_stock_return['R, N or W'] = monthly_stock_return['R, N or W'].ffill()
monthly_stock_return.dropna(inplace=True)
monthly_stock_return.head(10)

,Stkcd,month,monthly_stock_return,market_value,total_assets,total_shareholders_equity,BM ratio,S or B,"H, N or L",SMB_{B/M},ROE,"R, N or W"
5,000001,2000-06,0.007222,2.813499e+10,4.973234e+10,3.078513e+09,0.109419,B,H,BH,0.000000,W
6,000001,2000-07,0.020960,2.872469e+10,4.973234e+10,3.078513e+09,0.107173,B,H,BH,0.000000,W
7,000001,2000-08,-0.041059,2.754529e+10,4.973234e+10,3.078513e+09,0.111762,B,H,BH,0.000000,W
8,000001,2000-09,-0.044507,2.631933e+10,4.973234e+10,3.078513e+09,0.116968,B,H,BH,0.000000,W
9,000001,2000-10,0.034788,2.723492e+10,4.973234e+10,3.078513e+09,0.113036,B,H,BH,0.000000,W
10,000001,2000-11,0.010403,3.014079e+10,4.973234e+10,3.078513e+09,0.102138,B,H,BH,0.000000,W
11,000001,2000-12,-0.062621,2.825334e+10,6.722750e+10,4.738884e+09,0.167728,B,H,BH,0.106893,W
12,000001,2001-01,0.031680,2.914842e+10,6.600617e+10,3.517551e+09,0.120677,B,H,BH,0.131619,W
13,000001,2001-02,-0.059413,2.741663e+10,6.600617e+10,3.517551e+09,0.128300,B,H,BH,0.131619,W
14,000001,2001-03,0.151171,3.156124e+10,6.600617e+10,3.517551e+09,0.111452,B,H,BH,0.131619,W


In [104]:
monthly_stock_return['SMB_{OP}'] = monthly_stock_return['S or B'].astype(str) + monthly_stock_return[
    'R, N or W'].astype(str)  # combine the two tags
monthly_stock_return.head(10)

,Stkcd,month,monthly_stock_return,market_value,total_assets,total_shareholders_equity,BM ratio,S or B,"H, N or L",SMB_{B/M},ROE,"R, N or W",SMB_{OP}
5,000001,2000-06,0.007222,2.813499e+10,4.973234e+10,3.078513e+09,0.109419,B,H,BH,0.000000,W,BW
6,000001,2000-07,0.020960,2.872469e+10,4.973234e+10,3.078513e+09,0.107173,B,H,BH,0.000000,W,BW
7,000001,2000-08,-0.041059,2.754529e+10,4.973234e+10,3.078513e+09,0.111762,B,H,BH,0.000000,W,BW
8,000001,2000-09,-0.044507,2.631933e+10,4.973234e+10,3.078513e+09,0.116968,B,H,BH,0.000000,W,BW
9,000001,2000-10,0.034788,2.723492e+10,4.973234e+10,3.078513e+09,0.113036,B,H,BH,0.000000,W,BW
10,000001,2000-11,0.010403,3.014079e+10,4.973234e+10,3.078513e+09,0.102138,B,H,BH,0.000000,W,BW
11,000001,2000-12,-0.062621,2.825334e+10,6.722750e+10,4.738884e+09,0.167728,B,H,BH,0.106893,W,BW
12,000001,2001-01,0.031680,2.914842e+10,6.600617e+10,3.517551e+09,0.120677,B,H,BH,0.131619,W,BW
13,000001,2001-02,-0.059413,2.741663e+10,6.600617e+10,3.517551e+09,0.128300,B,H,BH,0.131619,W,BW
14,000001,2001-03,0.151171,3.156124e+10,6.600617e+10,3.517551e+09,0.111452,B,H,BH,0.131619,W,BW


In [105]:
temp = monthly_stock_return.groupby(['month', 'SMB_{OP}']).apply(
    lambda x: np.average(x['monthly_stock_return'], weights=x['market_value']), include_groups=True).unstack().fillna(0)
temp.columns = ['BN(of OP)', 'BR', 'BW', 'SN(of OP)', 'SR', 'SW']
source_group = pd.concat([source_group, temp], axis=1)
source_group.head(5)

/tmp/ipykernel_1676/4239516764.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  temp = monthly_stock_return.groupby(['month', 'SMB_{OP}']).apply(


,BH,BL,BN,Bnan,SH,SL,SN,SMB_{B/M},BN(of OP),BR,BW,SN(of OP),SR,SW
2000-01,0.211263,0.0,0.0,0.061891,0.115403,0.000000,0.096189,0.000110,0.414421,0.0,0.179034,0.041702,0.156997,0.094926
2000-02,0.199451,0.0,0.0,-0.011333,0.124140,0.000000,0.096888,0.007192,0.147461,0.0,0.154631,0.110890,0.069322,0.110437
2000-03,0.083746,0.0,0.0,0.002729,0.127617,0.125206,0.108970,0.092683,0.007946,0.0,0.080166,0.218151,0.313103,0.121351
2000-04,0.055933,0.0,0.0,0.037017,0.001406,0.000000,0.026939,-0.009196,0.036121,0.0,0.034367,0.031224,0.042017,0.007449
2000-05,0.032373,0.0,0.0,-0.055118,0.038882,0.053875,0.000000,0.020128,0.018637,0.0,0.042738,-0.030788,-0.104839,0.046624


In [106]:
# source_group.drop(columns=['BN', 'BR', 'BW', 'SN(of OP)', 'SR', 'SW'], inplace=True)

In [107]:
source_group['SMB_{OP}'] = (source_group['SR'] + source_group['SN(of OP)'] + source_group['SW']) / 3 - (
            source_group['BR'] + source_group['BN(of OP)'] + source_group['BW']) / 3

In [108]:
source_group.head(5)

,BH,BL,BN,Bnan,SH,SL,SN,SMB_{B/M},BN(of OP),BR,BW,SN(of OP),SR,SW,SMB_{OP}
2000-01,0.211263,0.0,0.0,0.061891,0.115403,0.000000,0.096189,0.000110,0.414421,0.0,0.179034,0.041702,0.156997,0.094926,-0.099943
2000-02,0.199451,0.0,0.0,-0.011333,0.124140,0.000000,0.096888,0.007192,0.147461,0.0,0.154631,0.110890,0.069322,0.110437,-0.003814
2000-03,0.083746,0.0,0.0,0.002729,0.127617,0.125206,0.108970,0.092683,0.007946,0.0,0.080166,0.218151,0.313103,0.121351,0.188164
2000-04,0.055933,0.0,0.0,0.037017,0.001406,0.000000,0.026939,-0.009196,0.036121,0.0,0.034367,0.031224,0.042017,0.007449,0.003400
2000-05,0.032373,0.0,0.0,-0.055118,0.038882,0.053875,0.000000,0.020128,0.018637,0.0,0.042738,-0.030788,-0.104839,0.046624,-0.050126


### now we can tag and calculate $SMB_{INV}$

In [109]:
def C_N_or_A(x):
    edge1 = x.quantile(.3)
    edge2 = x.quantile(.7)
    tmp = x.apply(lambda y: 'C' if y > edge2 else 'A' if y < edge1 else 'N')
    return tmp


balance_sheet['total_assets'] = balance_sheet['total_assets'].astype(np.float64)
balance_sheet['total_assets'] = balance_sheet['total_assets'].fillna(0)
balance_sheet['assets_increasing_rate'] = balance_sheet.groupby('Stkcd')[
    'total_assets'].pct_change()  # calculate the increasing rate of total assets
balance_sheet['assets_increasing_rate'] = balance_sheet['assets_increasing_rate'].replace([np.inf, -np.inf], np.nan)
balance_sheet.head(5)

,Stkcd,month,total_assets,total_shareholders_equity,net_income,ROE,"R, N or W",assets_increasing_rate
0,000001,2000-06,4.973234e+10,3.078513e+09,NaN,0.000000,W,NaN
1,000001,2000-12,6.722750e+10,4.738884e+09,506551785.0,0.106893,W,0.351786
2,000001,2001-01,6.600617e+10,3.517551e+09,462975563.0,0.131619,W,-0.018167
3,000001,2001-06,8.518143e+10,4.961824e+09,223211685.0,0.044986,W,0.290507
4,000001,2001-12,1.201270e+11,3.627669e+09,402360428.0,0.110914,W,0.410249


In [110]:
balance_sheet['C, N or A'] = balance_sheet.groupby("month")['assets_increasing_rate'].transform(
    C_N_or_A)  # divide the stocks into 3 groups  according to the total assets
balance_sheet['C, N or A'] = balance_sheet['C, N or A'].astype('category')

In [111]:
# merge SMB_{INV} information into monthly_stock_return
monthly_stock_return = monthly_stock_return.merge(
    balance_sheet[['Stkcd', 'month', 'assets_increasing_rate', 'C, N or A']],
    left_on=['Stkcd', 'month'], right_on=['Stkcd', 'month'], how='left')
monthly_stock_return['C, N or A'] = monthly_stock_return['C, N or A'].ffill()
monthly_stock_return['assets_increasing_rate'] = monthly_stock_return['assets_increasing_rate'].ffill()
monthly_stock_return['SMB_{INV}'] = monthly_stock_return['S or B'].astype(str) + monthly_stock_return[
    'C, N or A'].astype(str)  # combine the two tags
monthly_stock_return['SMB_{INV}'] = monthly_stock_return['SMB_{INV}'].astype('category')

In [112]:
monthly_stock_return['SMB_{INV}'] = monthly_stock_return['S or B'].astype(str) + monthly_stock_return[
    'C, N or A'].astype(str)  # combine the two tags
monthly_stock_return['SMB_{INV}'] = monthly_stock_return['SMB_{INV}'].astype('category')
monthly_stock_return.head(10)

,Stkcd,month,monthly_stock_return,market_value,total_assets,total_shareholders_equity,BM ratio,S or B,"H, N or L",SMB_{B/M},ROE,"R, N or W",SMB_{OP},assets_increasing_rate,"C, N or A",SMB_{INV}
0,000001,2000-06,0.007222,2.813499e+10,4.973234e+10,3.078513e+09,0.109419,B,H,BH,0.000000,W,BW,NaN,N,BN
1,000001,2000-07,0.020960,2.872469e+10,4.973234e+10,3.078513e+09,0.107173,B,H,BH,0.000000,W,BW,NaN,N,BN
2,000001,2000-08,-0.041059,2.754529e+10,4.973234e+10,3.078513e+09,0.111762,B,H,BH,0.000000,W,BW,NaN,N,BN
3,000001,2000-09,-0.044507,2.631933e+10,4.973234e+10,3.078513e+09,0.116968,B,H,BH,0.000000,W,BW,NaN,N,BN
4,000001,2000-10,0.034788,2.723492e+10,4.973234e+10,3.078513e+09,0.113036,B,H,BH,0.000000,W,BW,NaN,N,BN
5,000001,2000-11,0.010403,3.014079e+10,4.973234e+10,3.078513e+09,0.102138,B,H,BH,0.000000,W,BW,NaN,N,BN
6,000001,2000-12,-0.062621,2.825334e+10,6.722750e+10,4.738884e+09,0.167728,B,H,BH,0.106893,W,BW,0.351786,C,BC
7,000001,2001-01,0.031680,2.914842e+10,6.600617e+10,3.517551e+09,0.120677,B,H,BH,0.131619,W,BW,-0.018167,N,BN
8,000001,2001-02,-0.059413,2.741663e+10,6.600617e+10,3.517551e+09,0.128300,B,H,BH,0.131619,W,BW,-0.018167,N,BN
9,000001,2001-03,0.151171,3.156124e+10,6.600617e+10,3.517551e+09,0.111452,B,H,BH,0.131619,W,BW,-0.018167,N,BN


In [113]:
temp = monthly_stock_return.groupby(['month', 'SMB_{INV}']).apply(
    lambda x: np.average(x['monthly_stock_return'], weights=x['market_value']), include_groups=True).unstack().fillna(0)
temp.columns = ['BA', 'BC', 'BN(of INV)', 'SA', 'SC', 'SN(of INV)']
source_group = pd.concat([source_group, temp], axis=1)

/tmp/ipykernel_1676/1672904495.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  temp = monthly_stock_return.groupby(['month', 'SMB_{INV}']).apply(
/tmp/ipykernel_1676/1672904495.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  temp = monthly_stock_return.groupby(['month', 'SMB_{INV}']).apply(


In [114]:
source_group['SMB_{INV}'] = (source_group['SC'] + source_group['SN(of INV)'] + source_group['SA']) / 3 - (
            source_group['BC'] + source_group['BN(of INV)'] + source_group['BA']) / 3
source_group.head(10)

,BH,BL,BN,Bnan,SH,SL,SN,SMB_{B/M},BN(of OP),BR,...,SR,SW,SMB_{OP},BA,BC,BN(of INV),SA,SC,SN(of INV),SMB_{INV}
2000-01,0.211263,0.000000,0.0,0.061891,0.115403,0.000000,0.096189,0.000110,0.414421,0.000000,...,0.156997,0.094926,-0.099943,0.000000,0.000000,0.194215,0.000000,0.000000,0.094538,-0.033226
2000-02,0.199451,0.000000,0.0,-0.011333,0.124140,0.000000,0.096888,0.007192,0.147461,0.000000,...,0.069322,0.110437,-0.003814,0.000000,0.000000,0.154171,0.000000,0.000000,0.110332,-0.014613
2000-03,0.083746,0.000000,0.0,0.002729,0.127617,0.125206,0.108970,0.092683,0.007946,0.000000,...,0.313103,0.121351,0.188164,0.000000,0.000000,0.075823,0.000000,0.000000,0.123342,0.015840
2000-04,0.055933,0.000000,0.0,0.037017,0.001406,0.000000,0.026939,-0.009196,0.036121,0.000000,...,0.042017,0.007449,0.003400,0.000000,0.000000,0.034472,0.000000,0.000000,0.007898,-0.008858
2000-05,0.032373,0.000000,0.0,-0.055118,0.038882,0.053875,0.000000,0.020128,0.018637,0.000000,...,-0.104839,0.046624,-0.050126,0.000000,0.000000,0.041343,0.000000,0.000000,0.044945,0.001201
2000-06,0.064659,0.000000,0.0,0.000000,0.015243,0.000000,0.000000,-0.016472,0.027486,0.039617,...,0.000000,0.011999,-0.027768,0.071980,0.038223,0.038974,0.016244,0.016120,0.010886,-0.035309
2000-07,0.046614,0.000000,0.0,0.000000,0.060891,0.000000,0.000000,0.004759,0.021017,0.124836,...,0.000000,0.054799,-0.019314,0.032840,0.032515,0.056435,0.100631,0.040961,0.049152,0.022985
2000-08,-0.009001,0.000000,0.0,0.000000,0.013230,0.000000,0.018193,0.013475,0.041570,0.021028,...,0.000000,0.021994,-0.022882,0.002696,0.007337,-0.006100,0.040490,0.008693,0.018640,0.021296
2000-09,0.000000,-0.037499,0.0,0.000000,0.000000,0.000000,-0.033494,0.001335,-0.025806,-0.067506,...,0.000000,-0.033057,0.026507,-0.027291,-0.032474,-0.048079,-0.016839,-0.036828,-0.035286,0.006297
2000-10,0.022320,0.000000,0.0,0.000000,0.039125,0.000000,0.049018,0.021941,-0.000530,0.034356,...,0.000000,0.049952,0.000574,0.031539,0.000515,0.025433,0.037993,0.025937,0.054301,0.020248


### we can calculate all the five factors now

In [115]:
tmp = source_group.copy()
tmp.reset_index(inplace=True)
tmp.columns = ['month', ] + list(tmp.columns[1:])

In [116]:
pd.merge(tmp, mkt_risk_premium, left_on='month', right_on='month', how='left')
# tmp.head(5)

,month,BH,BL,BN,Bnan,SH,SL,SN,SMB_{B/M},BN(of OP),...,SW,SMB_{OP},BA,BC,BN(of INV),SA,SC,SN(of INV),SMB_{INV},mkt_risk_premium
0,2000-01,0.211263,0.000000,0.0,0.061891,0.115403,0.000000,0.096189,0.000110,0.414421,...,0.094926,-0.099943,0.000000,0.000000,0.194215,0.000000,0.000000,0.094538,-0.033226,0.159452
1,2000-02,0.199451,0.000000,0.0,-0.011333,0.124140,0.000000,0.096888,0.007192,0.147461,...,0.110437,-0.003814,0.000000,0.000000,0.154171,0.000000,0.000000,0.110332,-0.014613,0.121265
2,2000-03,0.083746,0.000000,0.0,0.002729,0.127617,0.125206,0.108970,0.092683,0.007946,...,0.121351,0.188164,0.000000,0.000000,0.075823,0.000000,0.000000,0.123342,0.015840,0.053877
3,2000-04,0.055933,0.000000,0.0,0.037017,0.001406,0.000000,0.026939,-0.009196,0.036121,...,0.007449,0.003400,0.000000,0.000000,0.034472,0.000000,0.000000,0.007898,-0.008858,0.011737
4,2000-05,0.032373,0.000000,0.0,-0.055118,0.038882,0.053875,0.000000,0.020128,0.018637,...,0.046624,-0.050126,0.000000,0.000000,0.041343,0.000000,0.000000,0.044945,0.001201,0.027313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,2023-09,0.000000,-0.065938,0.0,0.000000,-0.004402,0.000000,0.000000,0.020512,-0.020178,...,-0.004324,-0.081142,0.006729,-0.007207,-0.004966,-0.008652,-0.024627,-0.004105,-0.010647,-0.011713
285,2023-10,0.000000,-0.119236,0.0,0.000000,-0.010501,0.000000,0.000000,0.036245,-0.013697,...,-0.010846,0.093668,-0.014914,-0.005309,-0.017971,-0.019492,0.008747,-0.010773,0.005559,-0.022304
286,2023-11,0.000000,0.006153,0.0,0.000000,0.046746,0.000000,0.000000,0.013531,0.019602,...,0.061279,0.024950,0.013875,0.061863,0.005606,0.052727,0.040806,0.061425,0.024538,0.002600
287,2023-12,0.000000,-0.074707,0.0,0.000000,-0.031004,0.000000,0.000000,0.014568,-0.018984,...,-0.007548,0.008962,-0.022834,0.009859,-0.014181,-0.044135,-0.018416,-0.006958,-0.014118,-0.021379


In [117]:
source_group.reset_index(inplace=True)
source_group.columns = ['month', ] + list(source_group.columns[1:])
source_group = source_group.merge(mkt_risk_premium, left_on='month', right_on='month', how='left')
source_group['SMB'] = (source_group['SMB_{B/M}'] + source_group['SMB_{OP}'] + source_group['SMB_{INV}']) / 3
source_group['HML'] = (source_group['SH'] + source_group['BH']) / 2 - (source_group['SL'] + source_group['BL']) / 2
source_group['RMW'] = (source_group['SR'] + source_group['BR']) / 2 - (source_group['SW'] + source_group['BW']) / 2
source_group['CMA'] = (source_group['SC'] + source_group['BC']) / 2 - (source_group['SA'] + source_group['BA']) / 2
source_group.head(10)

,month,BH,BL,BN,Bnan,SH,SL,SN,SMB_{B/M},BN(of OP),...,BN(of INV),SA,SC,SN(of INV),SMB_{INV},mkt_risk_premium,SMB,HML,RMW,CMA
0,2000-01,0.211263,0.000000,0.0,0.061891,0.115403,0.000000,0.096189,0.000110,0.414421,...,0.194215,0.000000,0.000000,0.094538,-0.033226,0.159452,-0.044353,0.163333,-0.058482,0.000000
1,2000-02,0.199451,0.000000,0.0,-0.011333,0.124140,0.000000,0.096888,0.007192,0.147461,...,0.154171,0.000000,0.000000,0.110332,-0.014613,0.121265,-0.003745,0.161796,-0.097873,0.000000
2,2000-03,0.083746,0.000000,0.0,0.002729,0.127617,0.125206,0.108970,0.092683,0.007946,...,0.075823,0.000000,0.000000,0.123342,0.015840,0.053877,0.098895,0.043078,0.055793,0.000000
3,2000-04,0.055933,0.000000,0.0,0.037017,0.001406,0.000000,0.026939,-0.009196,0.036121,...,0.034472,0.000000,0.000000,0.007898,-0.008858,0.011737,-0.004885,0.028669,0.000100,0.000000
4,2000-05,0.032373,0.000000,0.0,-0.055118,0.038882,0.053875,0.000000,0.020128,0.018637,...,0.041343,0.000000,0.000000,0.044945,0.001201,0.027313,-0.009599,0.008690,-0.097100,0.000000
5,2000-06,0.064659,0.000000,0.0,0.000000,0.015243,0.000000,0.000000,-0.016472,0.027486,...,0.038974,0.016244,0.016120,0.010886,-0.035309,0.024876,-0.026516,0.039951,-0.006895,-0.016941
6,2000-07,0.046614,0.000000,0.0,0.000000,0.060891,0.000000,0.000000,0.004759,0.021017,...,0.056435,0.100631,0.040961,0.049152,0.022985,0.040949,0.002810,0.053753,0.008186,-0.029997
7,2000-08,-0.009001,0.000000,0.0,0.000000,0.013230,0.000000,0.018193,0.013475,0.041570,...,-0.006100,0.040490,0.008693,0.018640,0.021296,-0.006881,0.003963,0.002114,0.002432,-0.013578
8,2000-09,0.000000,-0.037499,0.0,0.000000,0.000000,0.000000,-0.033494,0.001335,-0.025806,...,-0.048079,-0.016839,-0.036828,-0.035286,0.006297,-0.046704,0.011380,0.018750,0.005768,-0.012586
9,2000-10,0.022320,0.000000,0.0,0.000000,0.039125,0.000000,0.049018,0.021941,-0.000530,...,0.025433,0.037993,0.025937,0.054301,0.020248,0.026483,0.014254,0.030723,-0.019950,-0.021540


In [118]:
# source_group.to_feather(r'temp/source_group1_in_part1.feather')  # save the result to a feather file

In [119]:
# now we have cleaned 5-factors
factors = source_group[['month', 'mkt_risk_premium', 'SMB', 'HML', 'RMW', 'CMA']]
factors.head(10)

,month,mkt_risk_premium,SMB,HML,RMW,CMA
0,2000-01,0.159452,-0.044353,0.163333,-0.058482,0.000000
1,2000-02,0.121265,-0.003745,0.161796,-0.097873,0.000000
2,2000-03,0.053877,0.098895,0.043078,0.055793,0.000000
3,2000-04,0.011737,-0.004885,0.028669,0.000100,0.000000
4,2000-05,0.027313,-0.009599,0.008690,-0.097100,0.000000
5,2000-06,0.024876,-0.026516,0.039951,-0.006895,-0.016941
6,2000-07,0.040949,0.002810,0.053753,0.008186,-0.029997
7,2000-08,-0.006881,0.003963,0.002114,0.002432,-0.013578
8,2000-09,-0.046704,0.011380,0.018750,0.005768,-0.012586
9,2000-10,0.026483,0.014254,0.030723,-0.019950,-0.021540


### calculate the excess return of each stock

the excess return of each stock is the return of the stock minus the risk-free rate, and this step is the same as the one in 1.3.2

In [120]:
"""
calculate the excess return of each stock
tmp_return is originally defined in 1.3.2
we just need to substitute 5-factor data from variable "factors"
"""
tmp_return.drop(columns=['risk_premium', 'smb', 'hml', 'rmw', 'cma'], inplace=True)
tmp_return.head(5)

,Stkcd,Trdmnt,Mretnd,daily_rf,excess_return
0,000001,2000-01,0.061891,0.000061,0.06183
1,000001,2000-02,-0.011333,0.000061,-0.011394
2,000001,2000-03,0.002729,0.000061,0.002668
3,000001,2000-04,0.037017,0.000061,0.036956
4,000001,2000-05,-0.055118,0.000061,-0.055179


In [121]:
tmp_return.shape

(730499, 5)

In [122]:
factors[factors['month'] == '2000-01'][['mkt_risk_premium', 'SMB', 'HML', 'RMW', 'CMA']]

,mkt_risk_premium,SMB,HML,RMW,CMA
0,0.159452,-0.044353,0.163333,-0.058482,0.0


In [123]:
# 3min20s
a = tmp_return['Trdmnt'].apply(
    lambda x: factors[factors['month'] == x][['mkt_risk_premium', 'SMB', 'HML', 'RMW', 'CMA']])
tmp_return.loc[:, ['mkt_risk_premium', 'SMB', 'HML', 'RMW', 'CMA']] = pd.concat(a.tolist(), ignore_index=True)
tmp_return.head(5)

,Stkcd,Trdmnt,Mretnd,daily_rf,excess_return,mkt_risk_premium,SMB,HML,RMW,CMA
0,000001,2000-01,0.061891,0.000061,0.06183,0.159452,-0.044353,0.163333,-0.058482,0.0
1,000001,2000-02,-0.011333,0.000061,-0.011394,0.121265,-0.003745,0.161796,-0.097873,0.0
2,000001,2000-03,0.002729,0.000061,0.002668,0.053877,0.098895,0.043078,0.055793,0.0
3,000001,2000-04,0.037017,0.000061,0.036956,0.011737,-0.004885,0.028669,0.000100,0.0
4,000001,2000-05,-0.055118,0.000061,-0.055179,0.027313,-0.009599,0.008690,-0.097100,0.0


In [124]:
# tmp_return.loc[:,['mkt_risk_premium', 'SMB', 'HML', 'RMW', 'CMA']] = pd.concat(a.tolist(), ignore_index=True)
# tmp_return.head(5)

In [125]:
def regress(data):
    X = data[['mkt_risk_premium', 'SMB', 'HML', 'RMW', 'CMA']]
    X = sm.add_constant(X)
    X.replace([np.inf, -np.inf], np.nan, inplace=True)
    X.fillna(0, inplace=True)
    y = data['excess_return']
    model = sm.OLS(np.array(y.astype(float)), X.astype(float))
    result = model.fit()
    params_and_tvalues = np.append(result.params, result.tvalues)
    return params_and_tvalues


betas = (tmp_return.groupby('Stkcd').apply(regress, include_groups=False)).apply(
    pd.Series)  # calculate the correlation between the stock return and the risk premium
betas.columns = ['const', 'risk_premium', 'smb', 'hml', 'rmw', 'cma', 't-const', 't-risk_premium', 't-smb', 't-hml',
                 't-rmw', 't-cma']

In [126]:
betas.head(5)

,const,risk_premium,smb,hml,rmw,cma,t-const,t-risk_premium,t-smb,t-hml,t-rmw,t-cma
Stkcd,,,,,,,,,,,,
000001,-0.001481,1.259903,-0.445768,-0.343795,-0.020177,-0.151694,-0.343498,14.974222,-5.284118,-2.848013,-0.601922,-2.824426
000002,0.003280,1.110387,-0.747936,-0.139013,0.090541,-0.083423,0.524907,9.116549,-6.152642,-0.783725,1.876791,-1.069751
000003,-0.064378,2.105664,1.824316,-0.651468,-0.876329,-2.763080,-2.300469,1.614795,2.278388,-0.507726,-1.476523,-1.251229
000004,0.012029,0.791428,0.724726,0.287249,0.070884,0.084147,1.683599,5.719125,5.244762,1.447363,1.291588,0.957360
000005,0.001017,0.955419,0.173242,0.756070,-0.135693,-0.112425,0.088055,4.257086,0.736959,2.351812,-1.372518,-0.784222


so in variable 'betas' above we can have the excess return of each single stock.

However, in Fama and French (2016), they use sorted portfolios to examine, (in Table 5 P9 in the paper), so we need to construct these portfolios.

we will do a mix-OLS regression to get the factor exposure.

In [128]:
X = tmp_return.dropna()[['mkt_risk_premium', 'SMB', 'HML', 'RMW', 'CMA']]
X = sm.add_constant(X)
Y = tmp_return.dropna()['excess_return']
model = sm.OLS(Y.astype(float), X.astype(float))
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          excess_return   R-squared:                       0.249
Model:                            OLS   Adj. R-squared:                  0.249
Method:                 Least Squares   F-statistic:                 4.768e+04
Date:                Sat, 22 Jun 2024   Prob (F-statistic):               0.00
Time:                        10:57:52   Log-Likelihood:             3.9387e+05
No. Observations:              720411   AIC:                        -7.877e+05
Df Residuals:                  720405   BIC:                        -7.877e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.0104      0.000     60.527      0.000       0.010       0.011
mkt_risk_premium     0.9420      0.003    300.843      0.000       0.936       0.948
SMB                  0.3970      0.004    108.816      0.000       0.390       0.404
HML                  0.1706      0.005     37.357      0.000       0.162       0.180
RMW                  0.0452      0.001     34.486      0.000       0.043       0.048
CMA                  0.0549      0.002     25.643      0.000       0.051       0.059
==============================================================================
Omnibus:                  1513825.001   Durbin-Watson:                   2.062
Prob(Omnibus):                  0.000   Jarque-Bera (JB):      65343045140.920
Skew:                          17.239   Prob(JB):                         0.00
Kurtosis:                    1478.015   Cond. No.                         31.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [129]:
with open(r'output/part1/1.4.1 mix_ols_summary1.html', 'w') as tar_file:
    tar_file.write(result.summary().as_html())
with open(r'output/part1/1.4.1 mix_ols_summary2.html', 'w') as tar_file:
    tar_file.write(result.summary2().as_html())

## GRS test

![grs.png](img/GRS_test.jpg)

firstly, we need to construct the portfolios

In [130]:
def five_division(x):
    edge1 = x.quantile(.2)
    edge2 = x.quantile(.4)
    edge3 = x.quantile(.6)
    edge4 = x.quantile(.8)
    tmp = x.apply(lambda y: '1' if y < edge1 else '2' if y < edge2 else '3' if y < edge3 else '4' if y < edge4 else '5')
    return tmp


def four_division(x):
    edge1 = x.quantile(.25)
    edge2 = x.quantile(.5)
    edge3 = x.quantile(.75)
    tmp = x.apply(lambda y: '1' if y < edge1 else '2' if y < edge2 else '3' if y < edge3 else '4')
    return tmp


def add_rank(monthly_stock_return):
    # add 25 size-BM portfolio rank, just like in Table 5 in the paper
    monthly_stock_return['ordered_size'] = monthly_stock_return.groupby('month')['market_value'].transform(
        lambda x: pd.qcut(x, 5, labels=["1", "2", "3", "4", "5"]))
    monthly_stock_return['ordered_BM'] = monthly_stock_return.groupby('month')['BM ratio'].transform(
        lambda x: pd.qcut(x, 5, labels=["1", "2", "3", "4", "5"]))
    monthly_stock_return['size-BM-rank'] = monthly_stock_return['ordered_size'].astype(str) + monthly_stock_return[
        'ordered_BM'].astype(str)
    monthly_stock_return['size-BM-rank'] = monthly_stock_return['size-BM-rank'].astype('category')

    # add 25 size-OP portfolio rank
    monthly_stock_return['ordered_OP'] = monthly_stock_return.groupby('month')['ROE'].transform(five_division)
    monthly_stock_return['size-OP-rank'] = monthly_stock_return['ordered_size'].astype(str) + monthly_stock_return[
        'ordered_OP'].astype(str)
    monthly_stock_return['size-OP-rank'] = monthly_stock_return['size-OP-rank'].astype('category')

    # add 25 size-INV portfolio rank
    monthly_stock_return['ordered_INV'] = monthly_stock_return.groupby('month')['assets_increasing_rate'].transform(
        five_division)
    monthly_stock_return['size-INV-rank'] = monthly_stock_return['ordered_size'].astype(str) + monthly_stock_return[
        'ordered_INV'].astype(str)
    monthly_stock_return['size-INV-rank'] = monthly_stock_return['size-INV-rank'].astype('category')

    # add 32 size-BM-OP portfolio rank
    monthly_stock_return['ordered_BM_for_32'] = monthly_stock_return.groupby('month')['BM ratio'].transform(
        four_division)
    monthly_stock_return['ordered_OP_for_32'] = monthly_stock_return.groupby('month')['ROE'].transform(four_division)
    monthly_stock_return['size-BM-OP-rank'] = monthly_stock_return['S or B'].astype(str) + monthly_stock_return[
        'ordered_BM_for_32'].astype(str) + monthly_stock_return['ordered_OP_for_32'].astype(str)
    monthly_stock_return['size-BM-OP-rank'] = monthly_stock_return['size-BM-OP-rank'].astype('category')

    # add 32 size-BM-INV portfolio rank
    monthly_stock_return['ordered_INV_for_32'] = monthly_stock_return.groupby('month')[
        'assets_increasing_rate'].transform(four_division)
    monthly_stock_return['size-BM-INV-rank'] = monthly_stock_return['S or B'].astype(str) + monthly_stock_return[
        'ordered_BM_for_32'].astype(str) + monthly_stock_return['ordered_INV_for_32'].astype(str)
    monthly_stock_return['size-BM-INV-rank'] = monthly_stock_return['size-BM-INV-rank'].astype('category')

    # add 32 size-OP-INV portfolio rank
    monthly_stock_return['size-OP-INV-rank'] = monthly_stock_return['S or B'].astype(str) + monthly_stock_return[
        'ordered_OP_for_32'].astype(str) + monthly_stock_return['ordered_INV_for_32'].astype(str)
    monthly_stock_return['size-OP-INV-rank'] = monthly_stock_return['size-OP-INV-rank'].astype('category')

    return monthly_stock_return


monthly_stock_return = add_rank(monthly_stock_return)
monthly_stock_return.head(10)

,Stkcd,month,monthly_stock_return,market_value,total_assets,total_shareholders_equity,BM ratio,S or B,"H, N or L",SMB_{B/M},...,ordered_OP,size-OP-rank,ordered_INV,size-INV-rank,ordered_BM_for_32,ordered_OP_for_32,size-BM-OP-rank,ordered_INV_for_32,size-BM-INV-rank,size-OP-INV-rank
0,000001,2000-06,0.007222,2.813499e+10,4.973234e+10,3.078513e+09,0.109419,B,H,BH,...,1,51,5,55,1,1,B11,4,B14,B14
1,000001,2000-07,0.020960,2.872469e+10,4.973234e+10,3.078513e+09,0.107173,B,H,BH,...,1,51,5,55,1,1,B11,4,B14,B14
2,000001,2000-08,-0.041059,2.754529e+10,4.973234e+10,3.078513e+09,0.111762,B,H,BH,...,1,51,5,55,1,1,B11,4,B14,B14
3,000001,2000-09,-0.044507,2.631933e+10,4.973234e+10,3.078513e+09,0.116968,B,H,BH,...,1,51,5,55,1,1,B11,4,B14,B14
4,000001,2000-10,0.034788,2.723492e+10,4.973234e+10,3.078513e+09,0.113036,B,H,BH,...,1,51,5,55,1,1,B11,4,B14,B14
5,000001,2000-11,0.010403,3.014079e+10,4.973234e+10,3.078513e+09,0.102138,B,H,BH,...,1,51,5,55,1,1,B11,4,B14,B14
6,000001,2000-12,-0.062621,2.825334e+10,6.722750e+10,4.738884e+09,0.167728,B,H,BH,...,5,55,5,55,1,4,B14,4,B14,B44
7,000001,2001-01,0.031680,2.914842e+10,6.600617e+10,3.517551e+09,0.120677,B,H,BH,...,5,55,1,51,1,4,B14,1,B11,B41
8,000001,2001-02,-0.059413,2.741663e+10,6.600617e+10,3.517551e+09,0.128300,B,H,BH,...,5,55,1,51,1,4,B14,1,B11,B41
9,000001,2001-03,0.151171,3.156124e+10,6.600617e+10,3.517551e+09,0.111452,B,H,BH,...,5,55,1,51,1,4,B14,1,B11,B41


In [131]:
rf_monthly.head()

,month,monthly_rf,rf
0,2000-01,0.001893,0.0225
1,2000-02,0.001771,0.0225
2,2000-03,0.001893,0.0225
3,2000-04,0.001832,0.0225
4,2000-05,0.001893,0.0225


In [132]:
# add rf to monthly_stock_return and get monthly_stock_excess_return
# a little time-consuming, about 2min30s
monthly_stock_return['rf'] = monthly_stock_return['month'].apply(
    lambda x: rf_monthly[rf_monthly['month'] == x]['monthly_rf'].values[0])
monthly_stock_return['monthly_stock_excess_return'] = monthly_stock_return['monthly_stock_return'] - \
                                                      monthly_stock_return['rf']
monthly_stock_return.head(10)

,Stkcd,month,monthly_stock_return,market_value,total_assets,total_shareholders_equity,BM ratio,S or B,"H, N or L",SMB_{B/M},...,ordered_INV,size-INV-rank,ordered_BM_for_32,ordered_OP_for_32,size-BM-OP-rank,ordered_INV_for_32,size-BM-INV-rank,size-OP-INV-rank,rf,monthly_stock_excess_return
0,000001,2000-06,0.007222,2.813499e+10,4.973234e+10,3.078513e+09,0.109419,B,H,BH,...,5,55,1,1,B11,4,B14,B14,0.001832,0.005390
1,000001,2000-07,0.020960,2.872469e+10,4.973234e+10,3.078513e+09,0.107173,B,H,BH,...,5,55,1,1,B11,4,B14,B14,0.001893,0.019067
2,000001,2000-08,-0.041059,2.754529e+10,4.973234e+10,3.078513e+09,0.111762,B,H,BH,...,5,55,1,1,B11,4,B14,B14,0.001893,-0.042952
3,000001,2000-09,-0.044507,2.631933e+10,4.973234e+10,3.078513e+09,0.116968,B,H,BH,...,5,55,1,1,B11,4,B14,B14,0.001832,-0.046339
4,000001,2000-10,0.034788,2.723492e+10,4.973234e+10,3.078513e+09,0.113036,B,H,BH,...,5,55,1,1,B11,4,B14,B14,0.001893,0.032895
5,000001,2000-11,0.010403,3.014079e+10,4.973234e+10,3.078513e+09,0.102138,B,H,BH,...,5,55,1,1,B11,4,B14,B14,0.001832,0.008571
6,000001,2000-12,-0.062621,2.825334e+10,6.722750e+10,4.738884e+09,0.167728,B,H,BH,...,5,55,1,4,B14,4,B14,B44,0.001893,-0.064514
7,000001,2001-01,0.031680,2.914842e+10,6.600617e+10,3.517551e+09,0.120677,B,H,BH,...,1,51,1,4,B14,1,B11,B41,0.001893,0.029787
8,000001,2001-02,-0.059413,2.741663e+10,6.600617e+10,3.517551e+09,0.128300,B,H,BH,...,1,51,1,4,B14,1,B11,B41,0.001709,-0.061122
9,000001,2001-03,0.151171,3.156124e+10,6.600617e+10,3.517551e+09,0.111452,B,H,BH,...,1,51,1,4,B14,1,B11,B41,0.001893,0.149278


In [133]:
# get portfolios' monthly value-weighted return
size_bm_excess_return = monthly_stock_return.groupby(['month', 'size-BM-rank'], observed=False).apply(
    lambda x: np.average(x['monthly_stock_excess_return'], weights=x['market_value']),
    include_groups=False).unstack().fillna(0)
size_op_excess_return = monthly_stock_return.groupby(['month', 'size-OP-rank'], observed=False).apply(
    lambda x: np.average(x['monthly_stock_excess_return'], weights=x['market_value']),
    include_groups=False).unstack().fillna(0)
size_inv_excess_return = monthly_stock_return.groupby(['month', 'size-INV-rank'], observed=False).apply(
    lambda x: np.average(x['monthly_stock_excess_return'], weights=x['market_value']),
    include_groups=False).unstack().fillna(0)
size_bm_op_excess_return = monthly_stock_return.groupby(['month', 'size-BM-OP-rank'], observed=False).apply(
    lambda x: np.average(x['monthly_stock_excess_return'], weights=x['market_value']),
    include_groups=False).unstack().fillna(0)
size_bm_inv_excess_return = monthly_stock_return.groupby(['month', 'size-BM-INV-rank'], observed=False).apply(
    lambda x: np.average(x['monthly_stock_excess_return'], weights=x['market_value']),
    include_groups=False).unstack().fillna(0)
size_op_inv_excess_return = monthly_stock_return.groupby(['month', 'size-OP-INV-rank'], observed=False).apply(
    lambda x: np.average(x['monthly_stock_excess_return'], weights=x['market_value']),
    include_groups=False).unstack().fillna(0)

In [134]:
"""
    GRS Test
    copy from https://github.com/SteffenGue/GRS_Test/blob/main/GRSTest.py
    I have checked the formula in this function, and it's correct.
"""


def grs_test(resid: np.ndarray, alpha: np.ndarray, factors: np.ndarray) -> tuple:
    """ 
        Perform the Gibbons, Ross and Shaken (1989) test.
        :param resid: Matrix of residuals from the OLS of size TxK.
        :param alpha: Vector of alphas from the OLS of size Kx1.
        :param factors: Matrix of factor returns of size TxJ.
        return Test statistic and pValue of the test statistic.
    """
    # Determine the time series and assets
    iT, iK = resid.shape

    # Determine the amount of risk factors
    iJ = factors.shape[1]

    # Input size checks
    assert alpha.shape == (iK, 1)
    assert factors.shape == (iT, iJ)

    # Covariance of the residuals, variables are in columns.
    mCov = np.cov(resid, rowvar=False)

    # Mean of excess returns of the risk factors
    vMuRF = np.nanmean(factors, axis=0)

    try:
        assert vMuRF.shape == (1, iJ)
    except AssertionError:
        vMuRF = vMuRF.reshape(1, iJ)

    # Duplicate this series for T timestamps
    mMuRF = np.repeat(vMuRF, iT, axis=0)

    # Test statistic
    mCovRF = (factors - mMuRF).T @ (factors - mMuRF) / (iT - 1)
    dTestStat = (iT / iK) * ((iT - iK - iJ) / (iT - iJ - 1)) * \
                (alpha.T @ (np.linalg.inv(mCov) @ alpha)) / \
                (1 + (vMuRF @ (np.linalg.inv(mCovRF) @ vMuRF.T)))

    pVal = 1 - f.cdf(dTestStat, iK, iT - iK - 1)

    return dTestStat, pVal

In [135]:
def regress(data, factors):
    X = factors
    X = sm.add_constant(X)
    X = X.values
    y = data
    model = sm.OLS(np.array(y.astype(float)), X.astype(float))
    result = model.fit()
    alpha = result.params[0]
    resid = result.resid
    return alpha, resid


def get_alpha_and_resid(data):
    alpha = pd.DataFrame(data.apply(lambda x: x[0]))
    resid = pd.DataFrame(data.apply(lambda x: x[1]))
    alpha.columns = ['alpha', ]
    alpha.reset_index(inplace=True)
    return alpha, resid

In [136]:
def get_grs_stat(excess_return, factors):
    grs = pd.DataFrame([[0, ] * 5] * 5, columns=['GRS', 'p-value of GRS', 'A|a|', 'A|a|/A|re|', 'A(a^2)/A(re^2)'],
                       index=["HML", "HML RMW", "HML CMA", "RMW CMA", "HML RMW CMA"],
                       dtype=np.float64)  # A(re) means average excess return

    ## 3 factors
    tmp = excess_return.apply(lambda x: regress(x, factors[['mkt_risk_premium', 'SMB', 'HML']]), axis=0)
    tmp1, resid = get_alpha_and_resid(tmp)
    tmp = grs_test(resid.to_numpy(), tmp1['alpha'].to_numpy().reshape(-1, 1),
                   factors[['mkt_risk_premium', 'SMB', 'HML']].to_numpy())
    grs.iloc[0, 0] = tmp[0][0][0]
    grs.iloc[0, 1] = tmp[1][0][0]
    grs.iloc[0, 2] = tmp1['alpha'].abs().mean()

    ## 4 factors: HML RMW
    tmp = excess_return.apply(lambda x: regress(x, factors[['mkt_risk_premium', 'SMB', 'HML', 'RMW']]), axis=0)
    tmp1, resid = get_alpha_and_resid(tmp)
    tmp = grs_test(resid.to_numpy(), tmp1['alpha'].to_numpy().reshape(-1, 1),
                   factors[['mkt_risk_premium', 'SMB', 'HML', 'RMW']].to_numpy())
    grs.iloc[1, 0] = tmp[0][0][0]
    grs.iloc[1, 1] = tmp[1][0][0]
    grs.iloc[1, 2] = tmp1['alpha'].abs().mean()

    ## 4 factors: HML CMA
    tmp = excess_return.apply(lambda x: regress(x, factors[['mkt_risk_premium', 'SMB', 'HML', 'CMA']]), axis=0)
    tmp1, resid = get_alpha_and_resid(tmp)
    tmp = grs_test(resid.to_numpy(), tmp1['alpha'].to_numpy().reshape(-1, 1),
                   factors[['mkt_risk_premium', 'SMB', 'HML', 'CMA']].to_numpy())
    grs.iloc[2, 0] = tmp[0][0][0]
    grs.iloc[2, 1] = tmp[1][0][0]
    grs.iloc[2, 2] = tmp1['alpha'].abs().mean()

    ## 4 factors: RMW CMA
    tmp = excess_return.apply(lambda x: regress(x, factors[['mkt_risk_premium', 'SMB', 'RMW', 'CMA']]), axis=0)
    tmp1, resid = get_alpha_and_resid(tmp)
    tmp = grs_test(resid.to_numpy(), tmp1['alpha'].to_numpy().reshape(-1, 1),
                   factors[['mkt_risk_premium', 'SMB', 'RMW', 'CMA']].to_numpy())
    grs.iloc[3, 0] = tmp[0][0][0]
    grs.iloc[3, 1] = tmp[1][0][0]
    grs.iloc[3, 2] = tmp1['alpha'].abs().mean()

    ## 5 factors
    tmp = excess_return.apply(lambda x: regress(x, factors[['mkt_risk_premium', 'SMB', 'HML', 'RMW', 'CMA']]), axis=0)
    tmp1, resid = get_alpha_and_resid(tmp)
    tmp = grs_test(resid.to_numpy(), tmp1['alpha'].to_numpy().reshape(-1, 1),
                   factors[['mkt_risk_premium', 'SMB', 'HML', 'RMW', 'CMA']].to_numpy())
    grs.iloc[4, 0] = tmp[0][0][0]
    grs.iloc[4, 1] = tmp[1][0][0]
    grs.iloc[4, 2] = tmp1['alpha'].abs().mean()

    excess_return['avg'] = excess_return.mean(axis=1)
    A_re = excess_return['avg'].mean()
    excess_return.drop(columns=['avg'], inplace=True)
    grs['A|a|/A|re|'] = grs['A|a|'] / A_re
    grs['A(a^2)/A(re^2)'] = grs['A|a|'] ** 2 / A_re ** 2

    return grs

In [137]:
factors = factors.dropna()

In [138]:
# get grs stat sheet
grs_size_bm = get_grs_stat(size_bm_excess_return[:-1], factors)
grs_size_op = get_grs_stat(size_op_excess_return[:-1], factors)
grs_size_inv = get_grs_stat(size_inv_excess_return[:-1], factors)
grs_size_bm_op = get_grs_stat(size_bm_op_excess_return[:-1], factors)
grs_size_bm_inv = get_grs_stat(size_bm_inv_excess_return[:-1], factors)
grs_size_op_inv = get_grs_stat(size_op_inv_excess_return[:-1], factors)

/tmp/ipykernel_1676/2627536053.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  excess_return['avg'] = excess_return.mean(axis=1)
/tmp/ipykernel_1676/2627536053.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  excess_return.drop(columns=['avg'], inplace=True)
/tmp/ipykernel_1676/2627536053.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

In [139]:
grs_size_bm

,GRS,p-value of GRS,A|a|,A|a|/A|re|,A(a^2)/A(re^2)
HML,11.578597,1.110223e-16,0.006585,1.155687,1.335613
HML RMW,11.581872,1.110223e-16,0.006593,1.157067,1.338804
HML CMA,11.666865,1.110223e-16,0.006562,1.151672,1.326349
RMW CMA,11.314804,1.110223e-16,0.006888,1.208934,1.461521
HML RMW CMA,11.678640,1.110223e-16,0.006568,1.152745,1.328821


In [140]:
grs_size_bm_inv

,GRS,p-value of GRS,A|a|,A|a|/A|re|,A(a^2)/A(re^2)
HML,12.625555,1.110223e-16,0.009911,1.452259,2.109055
HML RMW,12.653204,1.110223e-16,0.009900,1.450635,2.104341
HML CMA,13.253102,1.110223e-16,0.009407,1.378346,1.899838
RMW CMA,13.271238,1.110223e-16,0.009849,1.443198,2.082821
HML RMW CMA,13.261802,1.110223e-16,0.009399,1.377188,1.896647


In [141]:
grs_size_bm.to_excel(r'output/part1/grs_size_bm.xlsx')
grs_size_op.to_excel(r'output/part1/grs_size_op.xlsx')
grs_size_inv.to_excel(r'output/part1/grs_size_inv.xlsx')
grs_size_bm_op.to_excel(r'output/part1/grs_size_bm_op.xlsx')
grs_size_bm_inv.to_excel(r'output/part1/grs_size_bm_inv.xlsx')
grs_size_op_inv.to_excel(r'output/part1/grs_size_op_inv.xlsx')

## 1.4.2 Method 2: Construct the 5-factor model using 2*2 factors

In [142]:
monthly_stock_return.head(10)

,Stkcd,month,monthly_stock_return,market_value,total_assets,total_shareholders_equity,BM ratio,S or B,"H, N or L",SMB_{B/M},...,ordered_INV,size-INV-rank,ordered_BM_for_32,ordered_OP_for_32,size-BM-OP-rank,ordered_INV_for_32,size-BM-INV-rank,size-OP-INV-rank,rf,monthly_stock_excess_return
0,000001,2000-06,0.007222,2.813499e+10,4.973234e+10,3.078513e+09,0.109419,B,H,BH,...,5,55,1,1,B11,4,B14,B14,0.001832,0.005390
1,000001,2000-07,0.020960,2.872469e+10,4.973234e+10,3.078513e+09,0.107173,B,H,BH,...,5,55,1,1,B11,4,B14,B14,0.001893,0.019067
2,000001,2000-08,-0.041059,2.754529e+10,4.973234e+10,3.078513e+09,0.111762,B,H,BH,...,5,55,1,1,B11,4,B14,B14,0.001893,-0.042952
3,000001,2000-09,-0.044507,2.631933e+10,4.973234e+10,3.078513e+09,0.116968,B,H,BH,...,5,55,1,1,B11,4,B14,B14,0.001832,-0.046339
4,000001,2000-10,0.034788,2.723492e+10,4.973234e+10,3.078513e+09,0.113036,B,H,BH,...,5,55,1,1,B11,4,B14,B14,0.001893,0.032895
5,000001,2000-11,0.010403,3.014079e+10,4.973234e+10,3.078513e+09,0.102138,B,H,BH,...,5,55,1,1,B11,4,B14,B14,0.001832,0.008571
6,000001,2000-12,-0.062621,2.825334e+10,6.722750e+10,4.738884e+09,0.167728,B,H,BH,...,5,55,1,4,B14,4,B14,B44,0.001893,-0.064514
7,000001,2001-01,0.031680,2.914842e+10,6.600617e+10,3.517551e+09,0.120677,B,H,BH,...,1,51,1,4,B14,1,B11,B41,0.001893,0.029787
8,000001,2001-02,-0.059413,2.741663e+10,6.600617e+10,3.517551e+09,0.128300,B,H,BH,...,1,51,1,4,B14,1,B11,B41,0.001709,-0.061122
9,000001,2001-03,0.151171,3.156124e+10,6.600617e+10,3.517551e+09,0.111452,B,H,BH,...,1,51,1,4,B14,1,B11,B41,0.001893,0.149278


In [143]:
monthly_stock_return = monthly_stock_return[
    ['Stkcd', 'month', 'monthly_stock_return', 'market_value', 'total_assets', 'total_shareholders_equity', 'BM ratio',
     'ROE', 'assets_increasing_rate', 'rf', 'monthly_stock_excess_return']]
monthly_stock_return.head(10)

,Stkcd,month,monthly_stock_return,market_value,total_assets,total_shareholders_equity,BM ratio,ROE,assets_increasing_rate,rf,monthly_stock_excess_return
0,000001,2000-06,0.007222,2.813499e+10,4.973234e+10,3.078513e+09,0.109419,0.000000,NaN,0.001832,0.005390
1,000001,2000-07,0.020960,2.872469e+10,4.973234e+10,3.078513e+09,0.107173,0.000000,NaN,0.001893,0.019067
2,000001,2000-08,-0.041059,2.754529e+10,4.973234e+10,3.078513e+09,0.111762,0.000000,NaN,0.001893,-0.042952
3,000001,2000-09,-0.044507,2.631933e+10,4.973234e+10,3.078513e+09,0.116968,0.000000,NaN,0.001832,-0.046339
4,000001,2000-10,0.034788,2.723492e+10,4.973234e+10,3.078513e+09,0.113036,0.000000,NaN,0.001893,0.032895
5,000001,2000-11,0.010403,3.014079e+10,4.973234e+10,3.078513e+09,0.102138,0.000000,NaN,0.001832,0.008571
6,000001,2000-12,-0.062621,2.825334e+10,6.722750e+10,4.738884e+09,0.167728,0.106893,0.351786,0.001893,-0.064514
7,000001,2001-01,0.031680,2.914842e+10,6.600617e+10,3.517551e+09,0.120677,0.131619,-0.018167,0.001893,0.029787
8,000001,2001-02,-0.059413,2.741663e+10,6.600617e+10,3.517551e+09,0.128300,0.131619,-0.018167,0.001709,-0.061122
9,000001,2001-03,0.151171,3.156124e+10,6.600617e+10,3.517551e+09,0.111452,0.131619,-0.018167,0.001893,0.149278


In [144]:
def S_or_B(x):
    edge = x.quantile(.5)
    tmp = x.apply(lambda y: 'S' if y < edge else 'B')
    return tmp


def H_or_L(x):
    edge = x.quantile(.5)
    tmp = x.apply(lambda y: 'L' if y < edge else 'H')
    return tmp


def R_or_W(x):
    edge = x.quantile(.5)
    tmp = x.apply(lambda y: 'W' if y < edge else 'R')
    return tmp


def C_or_A(x):
    edge = x.quantile(.5)
    tmp = x.apply(lambda y: 'C' if y < edge else 'A')
    return tmp


monthly_stock_return['S or B'] = monthly_stock_return.groupby("month")['market_value'].transform(
    S_or_B)  # divide the stocks into 2 groups according to the market value
monthly_stock_return['H or L'] = monthly_stock_return.groupby("month")['BM ratio'].transform(H_or_L)
monthly_stock_return['R or W'] = monthly_stock_return.groupby("month")['ROE'].transform(R_or_W)
monthly_stock_return['C or A'] = monthly_stock_return.groupby("month")['assets_increasing_rate'].transform(C_or_A)
monthly_stock_return.head(10)

,Stkcd,month,monthly_stock_return,market_value,total_assets,total_shareholders_equity,BM ratio,ROE,assets_increasing_rate,rf,monthly_stock_excess_return,S or B,H or L,R or W,C or A
0,000001,2000-06,0.007222,2.813499e+10,4.973234e+10,3.078513e+09,0.109419,0.000000,NaN,0.001832,0.005390,B,L,W,A
1,000001,2000-07,0.020960,2.872469e+10,4.973234e+10,3.078513e+09,0.107173,0.000000,NaN,0.001893,0.019067,B,L,W,A
2,000001,2000-08,-0.041059,2.754529e+10,4.973234e+10,3.078513e+09,0.111762,0.000000,NaN,0.001893,-0.042952,B,L,W,A
3,000001,2000-09,-0.044507,2.631933e+10,4.973234e+10,3.078513e+09,0.116968,0.000000,NaN,0.001832,-0.046339,B,L,W,A
4,000001,2000-10,0.034788,2.723492e+10,4.973234e+10,3.078513e+09,0.113036,0.000000,NaN,0.001893,0.032895,B,L,W,A
5,000001,2000-11,0.010403,3.014079e+10,4.973234e+10,3.078513e+09,0.102138,0.000000,NaN,0.001832,0.008571,B,L,W,A
6,000001,2000-12,-0.062621,2.825334e+10,6.722750e+10,4.738884e+09,0.167728,0.106893,0.351786,0.001893,-0.064514,B,L,R,A
7,000001,2001-01,0.031680,2.914842e+10,6.600617e+10,3.517551e+09,0.120677,0.131619,-0.018167,0.001893,0.029787,B,L,R,C
8,000001,2001-02,-0.059413,2.741663e+10,6.600617e+10,3.517551e+09,0.128300,0.131619,-0.018167,0.001709,-0.061122,B,L,R,C
9,000001,2001-03,0.151171,3.156124e+10,6.600617e+10,3.517551e+09,0.111452,0.131619,-0.018167,0.001893,0.149278,B,L,R,C


In [145]:
monthly_stock_return['size-BM'] = monthly_stock_return['S or B'].astype(str) + monthly_stock_return['H or L'].astype(
    str)
monthly_stock_return['size-OP'] = monthly_stock_return['S or B'].astype(str) + monthly_stock_return['R or W'].astype(
    str)
monthly_stock_return['size-INV'] = monthly_stock_return['S or B'].astype(str) + monthly_stock_return['C or A'].astype(
    str)

monthly_stock_return['size-BM'] = monthly_stock_return['size-BM'].astype('category')
monthly_stock_return['size-OP'] = monthly_stock_return['size-OP'].astype('category')
monthly_stock_return['size-INV'] = monthly_stock_return['size-INV'].astype('category')

monthly_stock_return.head(10)

,Stkcd,month,monthly_stock_return,market_value,total_assets,total_shareholders_equity,BM ratio,ROE,assets_increasing_rate,rf,monthly_stock_excess_return,S or B,H or L,R or W,C or A,size-BM,size-OP,size-INV
0,000001,2000-06,0.007222,2.813499e+10,4.973234e+10,3.078513e+09,0.109419,0.000000,NaN,0.001832,0.005390,B,L,W,A,BL,BW,BA
1,000001,2000-07,0.020960,2.872469e+10,4.973234e+10,3.078513e+09,0.107173,0.000000,NaN,0.001893,0.019067,B,L,W,A,BL,BW,BA
2,000001,2000-08,-0.041059,2.754529e+10,4.973234e+10,3.078513e+09,0.111762,0.000000,NaN,0.001893,-0.042952,B,L,W,A,BL,BW,BA
3,000001,2000-09,-0.044507,2.631933e+10,4.973234e+10,3.078513e+09,0.116968,0.000000,NaN,0.001832,-0.046339,B,L,W,A,BL,BW,BA
4,000001,2000-10,0.034788,2.723492e+10,4.973234e+10,3.078513e+09,0.113036,0.000000,NaN,0.001893,0.032895,B,L,W,A,BL,BW,BA
5,000001,2000-11,0.010403,3.014079e+10,4.973234e+10,3.078513e+09,0.102138,0.000000,NaN,0.001832,0.008571,B,L,W,A,BL,BW,BA
6,000001,2000-12,-0.062621,2.825334e+10,6.722750e+10,4.738884e+09,0.167728,0.106893,0.351786,0.001893,-0.064514,B,L,R,A,BL,BR,BA
7,000001,2001-01,0.031680,2.914842e+10,6.600617e+10,3.517551e+09,0.120677,0.131619,-0.018167,0.001893,0.029787,B,L,R,C,BL,BR,BC
8,000001,2001-02,-0.059413,2.741663e+10,6.600617e+10,3.517551e+09,0.128300,0.131619,-0.018167,0.001709,-0.061122,B,L,R,C,BL,BR,BC
9,000001,2001-03,0.151171,3.156124e+10,6.600617e+10,3.517551e+09,0.111452,0.131619,-0.018167,0.001893,0.149278,B,L,R,C,BL,BR,BC


In [146]:
# get portfolios' monthly value-weighted return
tmp1 = monthly_stock_return.groupby(['month', 'size-BM'], observed=False).apply(
    lambda x: np.average(x['monthly_stock_excess_return'], weights=x['market_value']),
    include_groups=False).unstack().fillna(0)
tmp2 = monthly_stock_return.groupby(['month', 'size-OP'], observed=False).apply(
    lambda x: np.average(x['monthly_stock_excess_return'], weights=x['market_value']),
    include_groups=False).unstack().fillna(0)
tmp3 = monthly_stock_return.groupby(['month', 'size-INV'], observed=False).apply(
    lambda x: np.average(x['monthly_stock_excess_return'], weights=x['market_value']),
    include_groups=False).unstack().fillna(0)

In [147]:
source_group2 = pd.concat([tmp1, tmp2, tmp3], axis=1)
source_group2 = source_group2.merge(mkt_risk_premium, left_on='month', right_on='month', how='left')
source_group2

,month,BH,BL,SH,SL,BR,BW,SR,SW,BA,BC,SA,SC,mkt_risk_premium
0,2000-01,0.147049,0.202820,0.080137,0.111567,0.197070,0.066832,0.097973,0.062699,0.186513,0.225283,0.089672,0.104570,0.159452
1,2000-02,0.116985,0.160234,0.100729,0.121037,0.148453,0.242794,0.105898,0.126142,0.155230,0.135646,0.102197,0.134588,0.121265
2,2000-03,0.082099,0.072532,0.121409,0.120054,0.066202,0.254261,0.119059,0.131586,0.067845,0.109945,0.121373,0.118908,0.053877
3,2000-04,0.014927,0.036351,0.009243,0.002288,0.030402,0.075561,0.007093,0.003525,0.030084,0.046055,0.006186,0.008207,0.011737
4,2000-05,0.037959,0.039811,0.046452,0.037535,0.037740,0.076369,0.043352,0.041084,0.041149,0.029467,0.045271,0.034583,0.027313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,2023-09,-0.016306,-0.005846,-0.005601,-0.006845,-0.006147,0.003878,-0.005701,-0.004127,-0.005926,-0.009199,-0.005528,-0.011521,-0.011713
285,2023-10,-0.050380,-0.018557,-0.012007,-0.014206,-0.019283,-0.001460,-0.011971,-0.016723,-0.019114,-0.015479,-0.011925,-0.020337,-0.022304
286,2023-11,0.010675,0.005143,0.060557,0.043979,0.004982,0.021569,0.060079,0.051397,0.005165,0.007657,0.059910,0.056728,0.002600
287,2023-12,-0.039226,-0.014865,-0.008000,-0.033746,-0.014988,-0.029998,-0.008154,-0.041813,-0.015031,-0.024009,-0.008161,-0.046569,-0.021379


In [148]:
source_group2['SMB'] = (source_group2['SH'] + source_group2['SL'] + source_group2['SR'] + source_group2['SW'] +
                        source_group2['SC'] + source_group2['SA']) / 6 - (
                                   source_group2['BH'] + source_group2['BL'] + source_group2['BR'] + source_group2[
                               'BW'] + source_group2['BC'] + source_group2['BA']) / 6
source_group2['HML'] = (source_group2['SH'] + source_group2['BH']) / 2 - (source_group2['SL'] + source_group2['BL']) / 2
source_group2['RMW'] = (source_group2['SR'] + source_group2['BR']) / 2 - (source_group2['SW'] + source_group2['BW']) / 2
source_group2['CMA'] = (source_group2['SC'] + source_group2['BC']) / 2 - (source_group2['SA'] + source_group2['BA']) / 2
source_group2.head(5)

,month,BH,BL,SH,SL,BR,BW,SR,SW,BA,BC,SA,SC,mkt_risk_premium,SMB,HML,RMW,CMA
0,2000-01,0.147049,0.202820,0.080137,0.111567,0.197070,0.066832,0.097973,0.062699,0.186513,0.225283,0.089672,0.104570,0.159452,-0.079825,-0.043600,0.082756,0.026834
1,2000-02,0.116985,0.160234,0.100729,0.121037,0.148453,0.242794,0.105898,0.126142,0.155230,0.135646,0.102197,0.134588,0.121265,-0.044792,-0.031779,-0.057293,0.006404
2,2000-03,0.082099,0.072532,0.121409,0.120054,0.066202,0.254261,0.119059,0.131586,0.067845,0.109945,0.121373,0.118908,0.053877,0.013251,0.005461,-0.100293,0.019818
3,2000-04,0.014927,0.036351,0.009243,0.002288,0.030402,0.075561,0.007093,0.003525,0.030084,0.046055,0.006186,0.008207,0.011737,-0.032806,-0.007235,-0.020795,0.008996
4,2000-05,0.037959,0.039811,0.046452,0.037535,0.037740,0.076369,0.043352,0.041084,0.041149,0.029467,0.045271,0.034583,0.027313,-0.002370,0.003533,-0.018180,-0.011185


In [149]:
factors2 = source_group2[['month', 'mkt_risk_premium', 'SMB', 'HML', 'RMW', 'CMA']]
factors2

,month,mkt_risk_premium,SMB,HML,RMW,CMA
0,2000-01,0.159452,-0.079825,-0.043600,0.082756,0.026834
1,2000-02,0.121265,-0.044792,-0.031779,-0.057293,0.006404
2,2000-03,0.053877,0.013251,0.005461,-0.100293,0.019818
3,2000-04,0.011737,-0.032806,-0.007235,-0.020795,0.008996
4,2000-05,0.027313,-0.002370,0.003533,-0.018180,-0.011185
...,...,...,...,...,...,...
284,2023-09,-0.011713,0.000037,-0.004608,-0.005799,-0.004633
285,2023-10,-0.022304,0.006184,-0.014812,-0.006535,-0.002389
286,2023-11,0.002600,0.046243,0.011055,-0.003952,-0.000345
287,2023-12,-0.021379,-0.001388,0.000692,0.024334,-0.023693


### run regression

In [150]:
tmp_return.head(10)

,Stkcd,Trdmnt,Mretnd,daily_rf,excess_return,mkt_risk_premium,SMB,HML,RMW,CMA
0,000001,2000-01,0.061891,0.000061,0.06183,0.159452,-0.044353,0.163333,-0.058482,0.000000
1,000001,2000-02,-0.011333,0.000061,-0.011394,0.121265,-0.003745,0.161796,-0.097873,0.000000
2,000001,2000-03,0.002729,0.000061,0.002668,0.053877,0.098895,0.043078,0.055793,0.000000
3,000001,2000-04,0.037017,0.000061,0.036956,0.011737,-0.004885,0.028669,0.000100,0.000000
4,000001,2000-05,-0.055118,0.000061,-0.055179,0.027313,-0.009599,0.008690,-0.097100,0.000000
5,000001,2000-06,0.007222,0.000061,0.007161,0.024876,-0.026516,0.039951,-0.006895,-0.016941
6,000001,2000-07,0.02096,0.000061,0.020899,0.040949,0.002810,0.053753,0.008186,-0.029997
7,000001,2000-08,-0.041059,0.000061,-0.04112,-0.006881,0.003963,0.002114,0.002432,-0.013578
8,000001,2000-09,-0.044507,0.000061,-0.044568,-0.046704,0.011380,0.018750,0.005768,-0.012586
9,000001,2000-10,0.034788,0.000061,0.034727,0.026483,0.014254,0.030723,-0.019950,-0.021540


In [151]:
tmp_return.drop(columns=['mkt_risk_premium', 'SMB', 'HML', 'RMW', 'CMA'], inplace=True)
tmp_return.head(5)

,Stkcd,Trdmnt,Mretnd,daily_rf,excess_return
0,000001,2000-01,0.061891,0.000061,0.06183
1,000001,2000-02,-0.011333,0.000061,-0.011394
2,000001,2000-03,0.002729,0.000061,0.002668
3,000001,2000-04,0.037017,0.000061,0.036956
4,000001,2000-05,-0.055118,0.000061,-0.055179


In [152]:
a = tmp_return['Trdmnt'].apply(
    lambda x: factors2[factors2['month'] == x][['mkt_risk_premium', 'SMB', 'HML', 'RMW', 'CMA']])
tmp_return.loc[:, ['mkt_risk_premium', 'SMB', 'HML', 'RMW', 'CMA']] = pd.concat(a.tolist(), ignore_index=True)
tmp_return.head(5)

,Stkcd,Trdmnt,Mretnd,daily_rf,excess_return,mkt_risk_premium,SMB,HML,RMW,CMA
0,000001,2000-01,0.061891,0.000061,0.06183,0.159452,-0.079825,-0.043600,0.082756,0.026834
1,000001,2000-02,-0.011333,0.000061,-0.011394,0.121265,-0.044792,-0.031779,-0.057293,0.006404
2,000001,2000-03,0.002729,0.000061,0.002668,0.053877,0.013251,0.005461,-0.100293,0.019818
3,000001,2000-04,0.037017,0.000061,0.036956,0.011737,-0.032806,-0.007235,-0.020795,0.008996
4,000001,2000-05,-0.055118,0.000061,-0.055179,0.027313,-0.002370,0.003533,-0.018180,-0.011185


In [153]:
def regress(data):
    X = data[['mkt_risk_premium', 'SMB', 'HML', 'RMW', 'CMA']]
    X = sm.add_constant(X)
    X.replace([np.inf, -np.inf], np.nan, inplace=True)
    X.fillna(0, inplace=True)
    y = data['excess_return']
    model = sm.OLS(np.array(y.astype(float)), X.astype(float))
    result = model.fit()
    params_and_tvalues = np.append(result.params, result.tvalues)
    return params_and_tvalues


betas2 = (tmp_return.groupby('Stkcd').apply(regress, include_groups=False)).apply(
    pd.Series)  # calculate the correlation between the stock return and the risk premium
betas2.columns = ['const', 'risk_premium', 'smb', 'hml', 'rmw', 'cma', 't-const', 't-risk_premium', 't-smb', 't-hml',
                  't-rmw', 't-cma']
betas2.head(5)

,const,risk_premium,smb,hml,rmw,cma,t-const,t-risk_premium,t-smb,t-hml,t-rmw,t-cma
Stkcd,,,,,,,,,,,,
000001,0.000165,1.040982,-0.061866,-0.839370,0.034701,-0.007042,0.038626,18.716567,-0.960098,-5.758540,0.312924,-0.207578
000002,0.008066,0.878651,-0.110155,-1.060360,-0.112142,-0.074114,1.275887,10.493708,-1.157821,-4.909019,-0.683484,-1.479876
000003,-0.058041,1.889844,0.013053,3.358012,-0.695587,3.109231,-2.186803,4.340988,0.009440,1.140606,-0.918131,1.624251
000004,0.010873,0.940239,0.344005,1.034733,-0.570897,-0.016255,1.588938,10.670190,3.353449,4.445622,-3.241095,-0.302848
000005,0.004288,1.325810,-0.067198,2.325978,0.145979,0.037380,0.396932,9.385564,-0.415600,6.371722,0.521405,0.443842


we will do a mix-OLS regression to get the factor exposure.

In [154]:
X = tmp_return.dropna()[['mkt_risk_premium', 'SMB', 'HML', 'RMW', 'CMA']]
X = sm.add_constant(X)
Y = tmp_return.dropna()['excess_return']
model = sm.OLS(Y.astype(float), X.astype(float))
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          excess_return   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                 5.065e+04
Date:                Sat, 22 Jun 2024   Prob (F-statistic):               0.00
Time:                        11:03:31   Log-Likelihood:             3.9939e+05
No. Observations:              720411   AIC:                        -7.988e+05
Df Residuals:                  720405   BIC:                        -7.987e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.0104      0.000     62.124      0.000       0.010       0.011
mkt_risk_premium     1.0012      0.002    421.422      0.000       0.997       1.006
SMB                  0.1621      0.003     62.712      0.000       0.157       0.167
HML                  0.7346      0.006    126.690      0.000       0.723       0.746
RMW                 -0.2006      0.005    -43.706      0.000      -0.210      -0.192
CMA                  0.0155      0.001     11.215      0.000       0.013       0.018
==============================================================================
Omnibus:                  1526825.540   Durbin-Watson:                   2.065
Prob(Omnibus):                  0.000   Jarque-Bera (JB):      70205050297.565
Skew:                          17.587   Prob(JB):                         0.00
Kurtosis:                    1531.920   Cond. No.                         36.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [155]:
with open(r'output/part1/1.4.2 mix_ols_summary1.html', 'w') as tar_file:
    tar_file.write(result.summary().as_html())
with open(r'output/part1/1.4.2 mix_ols_summary2.html', 'w') as tar_file:
    tar_file.write(result.summary2().as_html())

## GRS test

In [156]:
monthly_stock_return.head(10)

,Stkcd,month,monthly_stock_return,market_value,total_assets,total_shareholders_equity,BM ratio,ROE,assets_increasing_rate,rf,monthly_stock_excess_return,S or B,H or L,R or W,C or A,size-BM,size-OP,size-INV
0,000001,2000-06,0.007222,2.813499e+10,4.973234e+10,3.078513e+09,0.109419,0.000000,NaN,0.001832,0.005390,B,L,W,A,BL,BW,BA
1,000001,2000-07,0.020960,2.872469e+10,4.973234e+10,3.078513e+09,0.107173,0.000000,NaN,0.001893,0.019067,B,L,W,A,BL,BW,BA
2,000001,2000-08,-0.041059,2.754529e+10,4.973234e+10,3.078513e+09,0.111762,0.000000,NaN,0.001893,-0.042952,B,L,W,A,BL,BW,BA
3,000001,2000-09,-0.044507,2.631933e+10,4.973234e+10,3.078513e+09,0.116968,0.000000,NaN,0.001832,-0.046339,B,L,W,A,BL,BW,BA
4,000001,2000-10,0.034788,2.723492e+10,4.973234e+10,3.078513e+09,0.113036,0.000000,NaN,0.001893,0.032895,B,L,W,A,BL,BW,BA
5,000001,2000-11,0.010403,3.014079e+10,4.973234e+10,3.078513e+09,0.102138,0.000000,NaN,0.001832,0.008571,B,L,W,A,BL,BW,BA
6,000001,2000-12,-0.062621,2.825334e+10,6.722750e+10,4.738884e+09,0.167728,0.106893,0.351786,0.001893,-0.064514,B,L,R,A,BL,BR,BA
7,000001,2001-01,0.031680,2.914842e+10,6.600617e+10,3.517551e+09,0.120677,0.131619,-0.018167,0.001893,0.029787,B,L,R,C,BL,BR,BC
8,000001,2001-02,-0.059413,2.741663e+10,6.600617e+10,3.517551e+09,0.128300,0.131619,-0.018167,0.001709,-0.061122,B,L,R,C,BL,BR,BC
9,000001,2001-03,0.151171,3.156124e+10,6.600617e+10,3.517551e+09,0.111452,0.131619,-0.018167,0.001893,0.149278,B,L,R,C,BL,BR,BC


In [157]:
monthly_stock_return = add_rank(monthly_stock_return)
monthly_stock_return.head(10)

,Stkcd,month,monthly_stock_return,market_value,total_assets,total_shareholders_equity,BM ratio,ROE,assets_increasing_rate,rf,...,ordered_OP,size-OP-rank,ordered_INV,size-INV-rank,ordered_BM_for_32,ordered_OP_for_32,size-BM-OP-rank,ordered_INV_for_32,size-BM-INV-rank,size-OP-INV-rank
0,000001,2000-06,0.007222,2.813499e+10,4.973234e+10,3.078513e+09,0.109419,0.000000,NaN,0.001832,...,1,51,5,55,1,1,B11,4,B14,B14
1,000001,2000-07,0.020960,2.872469e+10,4.973234e+10,3.078513e+09,0.107173,0.000000,NaN,0.001893,...,1,51,5,55,1,1,B11,4,B14,B14
2,000001,2000-08,-0.041059,2.754529e+10,4.973234e+10,3.078513e+09,0.111762,0.000000,NaN,0.001893,...,1,51,5,55,1,1,B11,4,B14,B14
3,000001,2000-09,-0.044507,2.631933e+10,4.973234e+10,3.078513e+09,0.116968,0.000000,NaN,0.001832,...,1,51,5,55,1,1,B11,4,B14,B14
4,000001,2000-10,0.034788,2.723492e+10,4.973234e+10,3.078513e+09,0.113036,0.000000,NaN,0.001893,...,1,51,5,55,1,1,B11,4,B14,B14
5,000001,2000-11,0.010403,3.014079e+10,4.973234e+10,3.078513e+09,0.102138,0.000000,NaN,0.001832,...,1,51,5,55,1,1,B11,4,B14,B14
6,000001,2000-12,-0.062621,2.825334e+10,6.722750e+10,4.738884e+09,0.167728,0.106893,0.351786,0.001893,...,5,55,5,55,1,4,B14,4,B14,B44
7,000001,2001-01,0.031680,2.914842e+10,6.600617e+10,3.517551e+09,0.120677,0.131619,-0.018167,0.001893,...,5,55,1,51,1,4,B14,1,B11,B41
8,000001,2001-02,-0.059413,2.741663e+10,6.600617e+10,3.517551e+09,0.128300,0.131619,-0.018167,0.001709,...,5,55,1,51,1,4,B14,1,B11,B41
9,000001,2001-03,0.151171,3.156124e+10,6.600617e+10,3.517551e+09,0.111452,0.131619,-0.018167,0.001893,...,5,55,1,51,1,4,B14,1,B11,B41


In [158]:
# get portfolios' monthly value-weighted return
size_bm_excess_return2 = monthly_stock_return.groupby(['month', 'size-BM-rank'], observed=False).apply(
    lambda x: np.average(x['monthly_stock_excess_return'], weights=x['market_value']),
    include_groups=False).unstack().fillna(0)
size_op_excess_return2 = monthly_stock_return.groupby(['month', 'size-OP-rank'], observed=False).apply(
    lambda x: np.average(x['monthly_stock_excess_return'], weights=x['market_value']),
    include_groups=False).unstack().fillna(0)
size_inv_excess_return2 = monthly_stock_return.groupby(['month', 'size-INV-rank'], observed=False).apply(
    lambda x: np.average(x['monthly_stock_excess_return'], weights=x['market_value']),
    include_groups=False).unstack().fillna(0)
size_bm_op_excess_return2 = monthly_stock_return.groupby(['month', 'size-BM-OP-rank'], observed=False).apply(
    lambda x: np.average(x['monthly_stock_excess_return'], weights=x['market_value']),
    include_groups=False).unstack().fillna(0)
size_bm_inv_excess_return2 = monthly_stock_return.groupby(['month', 'size-BM-INV-rank'], observed=False).apply(
    lambda x: np.average(x['monthly_stock_excess_return'], weights=x['market_value']),
    include_groups=False).unstack().fillna(0)
size_op_inv_excess_return2 = monthly_stock_return.groupby(['month', 'size-OP-INV-rank'], observed=False).apply(
    lambda x: np.average(x['monthly_stock_excess_return'], weights=x['market_value']),
    include_groups=False).unstack().fillna(0)

In [159]:
def regress(data, factors):
    X = factors
    X = sm.add_constant(X)
    X = X.values
    y = data
    model = sm.OLS(np.array(y.astype(float)), X.astype(float))
    result = model.fit()
    alpha = result.params[0]
    resid = result.resid
    return alpha, resid

In [160]:
factors2 = factors.dropna()

In [161]:
# get grs stat sheet
grs_size_bm2 = get_grs_stat(size_bm_excess_return2[:-1], factors2)
grs_size_op2 = get_grs_stat(size_op_excess_return2[:-1], factors2)
grs_size_inv2 = get_grs_stat(size_inv_excess_return2[:-1], factors2)
grs_size_bm_op2 = get_grs_stat(size_bm_op_excess_return2[:-1], factors2)
grs_size_bm_inv2 = get_grs_stat(size_bm_inv_excess_return2[:-1], factors2)
grs_size_op_inv2 = get_grs_stat(size_op_inv_excess_return2[:-1], factors2)

/tmp/ipykernel_1676/2627536053.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  excess_return['avg'] = excess_return.mean(axis=1)
/tmp/ipykernel_1676/2627536053.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  excess_return.drop(columns=['avg'], inplace=True)
/tmp/ipykernel_1676/2627536053.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

In [162]:
grs_size_bm2

,GRS,p-value of GRS,A|a|,A|a|/A|re|,A(a^2)/A(re^2)
HML,11.578597,1.110223e-16,0.006585,1.155687,1.335613
HML RMW,11.581872,1.110223e-16,0.006593,1.157067,1.338804
HML CMA,11.666865,1.110223e-16,0.006562,1.151672,1.326349
RMW CMA,11.314804,1.110223e-16,0.006888,1.208934,1.461521
HML RMW CMA,11.678640,1.110223e-16,0.006568,1.152745,1.328821


In [163]:
grs_size_bm_op2

,GRS,p-value of GRS,A|a|,A|a|/A|re|,A(a^2)/A(re^2)
HML,11.554180,1.110223e-16,0.006844,1.159950,1.345484
HML RMW,11.557575,1.110223e-16,0.006833,1.158082,1.341154
HML CMA,11.435948,1.110223e-16,0.006859,1.162457,1.351307
RMW CMA,11.355886,1.110223e-16,0.007118,1.206300,1.455159
HML RMW CMA,11.438828,1.110223e-16,0.006851,1.161008,1.347939


In [164]:
grs_size_bm2.to_excel(r'output/part1/grs_size_bm2.xlsx')
grs_size_op2.to_excel(r'output/part1/grs_size_op2.xlsx')
grs_size_inv2.to_excel(r'output/part1/grs_size_inv2.xlsx')
grs_size_bm_op2.to_excel(r'output/part1/grs_size_bm_op2.xlsx')
grs_size_bm_inv2.to_excel(r'output/part1/grs_size_bm_inv2.xlsx')
grs_size_op_inv2.to_excel(r'output/part1/grs_size_op_inv2.xlsx')

## 1.4.3 Method 3: Construct the 5-factor model using 2\*2\*2\*2 factors

In [165]:
monthly_stock_return.head(10)

,Stkcd,month,monthly_stock_return,market_value,total_assets,total_shareholders_equity,BM ratio,ROE,assets_increasing_rate,rf,...,ordered_OP,size-OP-rank,ordered_INV,size-INV-rank,ordered_BM_for_32,ordered_OP_for_32,size-BM-OP-rank,ordered_INV_for_32,size-BM-INV-rank,size-OP-INV-rank
0,000001,2000-06,0.007222,2.813499e+10,4.973234e+10,3.078513e+09,0.109419,0.000000,NaN,0.001832,...,1,51,5,55,1,1,B11,4,B14,B14
1,000001,2000-07,0.020960,2.872469e+10,4.973234e+10,3.078513e+09,0.107173,0.000000,NaN,0.001893,...,1,51,5,55,1,1,B11,4,B14,B14
2,000001,2000-08,-0.041059,2.754529e+10,4.973234e+10,3.078513e+09,0.111762,0.000000,NaN,0.001893,...,1,51,5,55,1,1,B11,4,B14,B14
3,000001,2000-09,-0.044507,2.631933e+10,4.973234e+10,3.078513e+09,0.116968,0.000000,NaN,0.001832,...,1,51,5,55,1,1,B11,4,B14,B14
4,000001,2000-10,0.034788,2.723492e+10,4.973234e+10,3.078513e+09,0.113036,0.000000,NaN,0.001893,...,1,51,5,55,1,1,B11,4,B14,B14
5,000001,2000-11,0.010403,3.014079e+10,4.973234e+10,3.078513e+09,0.102138,0.000000,NaN,0.001832,...,1,51,5,55,1,1,B11,4,B14,B14
6,000001,2000-12,-0.062621,2.825334e+10,6.722750e+10,4.738884e+09,0.167728,0.106893,0.351786,0.001893,...,5,55,5,55,1,4,B14,4,B14,B44
7,000001,2001-01,0.031680,2.914842e+10,6.600617e+10,3.517551e+09,0.120677,0.131619,-0.018167,0.001893,...,5,55,1,51,1,4,B14,1,B11,B41
8,000001,2001-02,-0.059413,2.741663e+10,6.600617e+10,3.517551e+09,0.128300,0.131619,-0.018167,0.001709,...,5,55,1,51,1,4,B14,1,B11,B41
9,000001,2001-03,0.151171,3.156124e+10,6.600617e+10,3.517551e+09,0.111452,0.131619,-0.018167,0.001893,...,5,55,1,51,1,4,B14,1,B11,B41


In [166]:
monthly_stock_return = monthly_stock_return[
    ['Stkcd', 'month', 'monthly_stock_return', 'market_value', 'total_assets', 'total_shareholders_equity', 'BM ratio',
     'ROE', 'assets_increasing_rate', 'rf', 'monthly_stock_excess_return', 'S or B', 'H or L', 'R or W', 'C or A']]
monthly_stock_return.head(10)

,Stkcd,month,monthly_stock_return,market_value,total_assets,total_shareholders_equity,BM ratio,ROE,assets_increasing_rate,rf,monthly_stock_excess_return,S or B,H or L,R or W,C or A
0,000001,2000-06,0.007222,2.813499e+10,4.973234e+10,3.078513e+09,0.109419,0.000000,NaN,0.001832,0.005390,B,L,W,A
1,000001,2000-07,0.020960,2.872469e+10,4.973234e+10,3.078513e+09,0.107173,0.000000,NaN,0.001893,0.019067,B,L,W,A
2,000001,2000-08,-0.041059,2.754529e+10,4.973234e+10,3.078513e+09,0.111762,0.000000,NaN,0.001893,-0.042952,B,L,W,A
3,000001,2000-09,-0.044507,2.631933e+10,4.973234e+10,3.078513e+09,0.116968,0.000000,NaN,0.001832,-0.046339,B,L,W,A
4,000001,2000-10,0.034788,2.723492e+10,4.973234e+10,3.078513e+09,0.113036,0.000000,NaN,0.001893,0.032895,B,L,W,A
5,000001,2000-11,0.010403,3.014079e+10,4.973234e+10,3.078513e+09,0.102138,0.000000,NaN,0.001832,0.008571,B,L,W,A
6,000001,2000-12,-0.062621,2.825334e+10,6.722750e+10,4.738884e+09,0.167728,0.106893,0.351786,0.001893,-0.064514,B,L,R,A
7,000001,2001-01,0.031680,2.914842e+10,6.600617e+10,3.517551e+09,0.120677,0.131619,-0.018167,0.001893,0.029787,B,L,R,C
8,000001,2001-02,-0.059413,2.741663e+10,6.600617e+10,3.517551e+09,0.128300,0.131619,-0.018167,0.001709,-0.061122,B,L,R,C
9,000001,2001-03,0.151171,3.156124e+10,6.600617e+10,3.517551e+09,0.111452,0.131619,-0.018167,0.001893,0.149278,B,L,R,C


In [167]:
monthly_stock_return['tag'] = monthly_stock_return['S or B'].astype(str) + monthly_stock_return['H or L'].astype(str) + \
                              monthly_stock_return['R or W'].astype(str) + monthly_stock_return['C or A'].astype(str)
monthly_stock_return['tag'] = monthly_stock_return['tag'].astype('category')
monthly_stock_return.head(10)

,Stkcd,month,monthly_stock_return,market_value,total_assets,total_shareholders_equity,BM ratio,ROE,assets_increasing_rate,rf,monthly_stock_excess_return,S or B,H or L,R or W,C or A,tag
0,000001,2000-06,0.007222,2.813499e+10,4.973234e+10,3.078513e+09,0.109419,0.000000,NaN,0.001832,0.005390,B,L,W,A,BLWA
1,000001,2000-07,0.020960,2.872469e+10,4.973234e+10,3.078513e+09,0.107173,0.000000,NaN,0.001893,0.019067,B,L,W,A,BLWA
2,000001,2000-08,-0.041059,2.754529e+10,4.973234e+10,3.078513e+09,0.111762,0.000000,NaN,0.001893,-0.042952,B,L,W,A,BLWA
3,000001,2000-09,-0.044507,2.631933e+10,4.973234e+10,3.078513e+09,0.116968,0.000000,NaN,0.001832,-0.046339,B,L,W,A,BLWA
4,000001,2000-10,0.034788,2.723492e+10,4.973234e+10,3.078513e+09,0.113036,0.000000,NaN,0.001893,0.032895,B,L,W,A,BLWA
5,000001,2000-11,0.010403,3.014079e+10,4.973234e+10,3.078513e+09,0.102138,0.000000,NaN,0.001832,0.008571,B,L,W,A,BLWA
6,000001,2000-12,-0.062621,2.825334e+10,6.722750e+10,4.738884e+09,0.167728,0.106893,0.351786,0.001893,-0.064514,B,L,R,A,BLRA
7,000001,2001-01,0.031680,2.914842e+10,6.600617e+10,3.517551e+09,0.120677,0.131619,-0.018167,0.001893,0.029787,B,L,R,C,BLRC
8,000001,2001-02,-0.059413,2.741663e+10,6.600617e+10,3.517551e+09,0.128300,0.131619,-0.018167,0.001709,-0.061122,B,L,R,C,BLRC
9,000001,2001-03,0.151171,3.156124e+10,6.600617e+10,3.517551e+09,0.111452,0.131619,-0.018167,0.001893,0.149278,B,L,R,C,BLRC


In [168]:
source_group3 = monthly_stock_return.groupby(['month', 'tag'], observed=False).apply(
    lambda x: np.average(x['monthly_stock_excess_return'], weights=x['market_value']),
    include_groups=False).unstack().fillna(0)
source_group3.head(10)

tag,BHRA,BHRC,BHWA,BHWC,BLRA,BLRC,BLWA,BLWC,SHRA,SHRC,SHWA,SHWC,SLRA,SLRC,SLWA,SLWC
month,,,,,,,,,,,,,,,,
2000-01,0.147049,0.000000,0.0,0.000000,0.199787,0.257489,0.143662,-0.043197,0.080137,0.0,0.0,0.0,0.136294,0.137656,0.090866,0.037567
2000-02,0.116985,0.000000,0.0,0.000000,0.164495,0.112289,0.203787,0.281882,0.100729,0.0,0.0,0.0,0.098263,0.134833,0.119656,0.134048
2000-03,0.082099,0.000000,0.0,0.000000,0.055054,0.097529,0.307785,0.192646,0.121409,0.0,0.0,0.0,0.113800,0.112172,0.132000,0.131130
2000-04,0.014927,0.000000,0.0,0.000000,0.030610,0.051291,0.123602,0.011300,0.009243,0.0,0.0,0.0,-0.001552,0.003967,-0.009798,0.016690
2000-05,0.037959,0.000000,0.0,0.000000,0.040649,0.022425,0.075858,0.077092,0.046452,0.0,0.0,0.0,0.035349,0.035725,0.050736,0.032421
2000-06,0.023833,0.000000,0.0,0.000000,0.040616,0.005993,0.042591,0.142844,0.008500,0.0,0.0,0.0,0.020655,-0.016917,0.013257,0.020816
2000-07,0.044256,0.000000,0.0,0.000000,0.052692,0.041479,0.055442,0.016147,0.050073,0.0,0.0,0.0,0.039244,0.089872,0.040835,0.108660
2000-08,-0.006850,0.100954,0.0,0.000000,-0.004863,-0.000674,-0.017211,-0.007033,0.023576,0.0,0.0,0.0,-0.028580,0.033837,0.012115,0.044495
2000-09,-0.045332,-0.069276,0.0,-0.009084,-0.047636,-0.037778,-0.058883,0.012821,-0.034705,0.0,0.0,0.0,-0.048570,-0.027403,-0.045273,0.001431


In [169]:
source_group3 = source_group3.merge(mkt_risk_premium, left_on='month', right_on='month', how='left')
source_group3

,month,BHRA,BHRC,BHWA,BHWC,BLRA,BLRC,BLWA,BLWC,SHRA,SHRC,SHWA,SHWC,SLRA,SLRC,SLWA,SLWC,mkt_risk_premium
0,2000-01,0.147049,0.000000,0.000000,0.000000,0.199787,0.257489,0.143662,-0.043197,0.080137,0.000000,0.0,0.000000,0.136294,0.137656,0.090866,0.037567,0.159452
1,2000-02,0.116985,0.000000,0.000000,0.000000,0.164495,0.112289,0.203787,0.281882,0.100729,0.000000,0.0,0.000000,0.098263,0.134833,0.119656,0.134048,0.121265
2,2000-03,0.082099,0.000000,0.000000,0.000000,0.055054,0.097529,0.307785,0.192646,0.121409,0.000000,0.0,0.000000,0.113800,0.112172,0.132000,0.131130,0.053877
3,2000-04,0.014927,0.000000,0.000000,0.000000,0.030610,0.051291,0.123602,0.011300,0.009243,0.000000,0.0,0.000000,-0.001552,0.003967,-0.009798,0.016690,0.011737
4,2000-05,0.037959,0.000000,0.000000,0.000000,0.040649,0.022425,0.075858,0.077092,0.046452,0.000000,0.0,0.000000,0.035349,0.035725,0.050736,0.032421,0.027313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,2023-09,-0.008099,-0.042287,-0.026501,-0.041017,-0.005934,-0.013107,0.000741,0.031810,-0.005612,0.000000,0.0,0.015335,-0.025252,0.005339,0.011206,-0.020898,-0.011713
285,2023-10,-0.035969,-0.135064,-0.047768,-0.072822,-0.019083,-0.009031,0.004069,0.017950,-0.012009,0.000000,0.0,-0.008679,0.006854,-0.031627,-0.018620,-0.015198,-0.022304
286,2023-11,0.015910,0.008478,0.021456,-0.042754,0.005067,-0.007515,-0.000098,0.073568,0.060589,0.000000,0.0,-0.005709,0.029051,0.030681,0.030352,0.071920,0.002600
287,2023-12,-0.027073,-0.074113,-0.024485,-0.068197,-0.014764,-0.009059,-0.033656,-0.018698,-0.007990,-0.029758,0.0,0.000000,-0.013908,-0.027817,-0.021039,-0.052303,-0.021379


In [170]:
source_group3['SMB'] = (source_group3['SHRC'] + source_group3['SHRA'] + source_group3['SHWC'] + source_group3['SHWA'] +
                        source_group3['SLRC'] + source_group3['SLRA'] + source_group3['SLWC'] + source_group3[
                            'SLWA']) / 8 - (source_group3['BHRC'] + source_group3['BHRA'] + source_group3['BHWC'] +
                                            source_group3['BHWA'] + source_group3['BLRC'] + source_group3['BLRA'] +
                                            source_group3['BLWC'] + source_group3['BLWA']) / 8
source_group3['HML'] = (source_group3['SHRC'] + source_group3['SHRA'] + source_group3['SHWC'] + source_group3['SHWA'] +
                        source_group3['BHRC'] + source_group3['BHRA'] + source_group3['BHWC'] + source_group3[
                            'BHWA']) / 8 - (source_group3['SLRC'] + source_group3['SLRA'] + source_group3['SLWC'] +
                                            source_group3['SLWA'] + source_group3['BLRC'] + source_group3['BLRA'] +
                                            source_group3['BLWC'] + source_group3['BLWA']) / 8
source_group3['RMW'] = (source_group3['SHRC'] + source_group3['SHRA'] + source_group3['SLRC'] + source_group3['SLRA'] +
                        source_group3['BHRC'] + source_group3['BHRA'] + source_group3['BLRC'] + source_group3[
                            'BLRA']) / 8 - (source_group3['SHWC'] + source_group3['SHWA'] + source_group3['SLWC'] +
                                            source_group3['SLWA'] + source_group3['BHWC'] + source_group3['BHWA'] +
                                            source_group3['BLWC'] + source_group3['BLWA']) / 8
source_group3['CMA'] = (source_group3['SHRC'] + source_group3['SHWC'] + source_group3['SLRC'] + source_group3['SLWC'] +
                        source_group3['BHRC'] + source_group3['BHWC'] + source_group3['BLRC'] + source_group3[
                            'BLWC']) / 8 - (source_group3['SHRA'] + source_group3['SHWA'] + source_group3['SLRA'] +
                                            source_group3['SLWA'] + source_group3['BHRA'] + source_group3['BHWA'] +
                                            source_group3['BLRA'] + source_group3['BLWA']) / 8

In [171]:
factors3 = source_group3[['month', 'mkt_risk_premium', 'SMB', 'HML', 'RMW', 'CMA']]
factors3

,month,mkt_risk_premium,SMB,HML,RMW,CMA
0,2000-01,0.159452,-0.027784,-0.091617,0.091189,-0.051035
1,2000-02,0.121265,-0.036489,-0.128943,-0.001473,-0.017608
2,2000-03,0.053877,-0.015575,-0.117326,-0.022687,-0.034834
3,2000-04,0.011737,-0.026647,-0.025242,-0.004163,-0.010473
4,2000-05,0.027313,-0.006662,-0.035731,-0.002193,-0.014917
...,...,...,...,...,...,...
284,2023-09,-0.011713,0.010564,-0.011511,-0.008204,-0.000672
285,2023-10,-0.022304,0.027305,-0.030953,-0.011858,-0.016493
286,2023-11,0.002600,0.017847,-0.021882,-0.000809,-0.004207
287,2023-12,-0.021379,0.014654,-0.005046,0.001737,-0.017129


### run regression

In [172]:
tmp_return.head(10)

,Stkcd,Trdmnt,Mretnd,daily_rf,excess_return,mkt_risk_premium,SMB,HML,RMW,CMA
0,000001,2000-01,0.061891,0.000061,0.06183,0.159452,-0.079825,-0.043600,0.082756,0.026834
1,000001,2000-02,-0.011333,0.000061,-0.011394,0.121265,-0.044792,-0.031779,-0.057293,0.006404
2,000001,2000-03,0.002729,0.000061,0.002668,0.053877,0.013251,0.005461,-0.100293,0.019818
3,000001,2000-04,0.037017,0.000061,0.036956,0.011737,-0.032806,-0.007235,-0.020795,0.008996
4,000001,2000-05,-0.055118,0.000061,-0.055179,0.027313,-0.002370,0.003533,-0.018180,-0.011185
5,000001,2000-06,0.007222,0.000061,0.007161,0.024876,-0.033887,-0.012427,-0.013126,0.021462
6,000001,2000-07,0.02096,0.000061,0.020899,0.040949,0.015974,-0.008325,-0.005788,0.016784
7,000001,2000-08,-0.041059,0.000061,-0.04112,-0.006881,0.028814,0.006286,0.002745,0.017103
8,000001,2000-09,-0.044507,0.000061,-0.044568,-0.046704,0.011430,0.001272,-0.002271,0.034320
9,000001,2000-10,0.034788,0.000061,0.034727,0.026483,0.018137,0.016176,-0.004000,-0.004578


In [173]:
tmp_return.drop(columns=['mkt_risk_premium', 'SMB', 'HML', 'RMW', 'CMA'], inplace=True)
tmp_return.head(5)

,Stkcd,Trdmnt,Mretnd,daily_rf,excess_return
0,000001,2000-01,0.061891,0.000061,0.06183
1,000001,2000-02,-0.011333,0.000061,-0.011394
2,000001,2000-03,0.002729,0.000061,0.002668
3,000001,2000-04,0.037017,0.000061,0.036956
4,000001,2000-05,-0.055118,0.000061,-0.055179


In [174]:
a = tmp_return['Trdmnt'].apply(
    lambda x: factors3[factors3['month'] == x][['mkt_risk_premium', 'SMB', 'HML', 'RMW', 'CMA']])
tmp_return.loc[:, ['mkt_risk_premium', 'SMB', 'HML', 'RMW', 'CMA']] = pd.concat(a.tolist(), ignore_index=True)
tmp_return.head(5)

,Stkcd,Trdmnt,Mretnd,daily_rf,excess_return,mkt_risk_premium,SMB,HML,RMW,CMA
0,000001,2000-01,0.061891,0.000061,0.06183,0.159452,-0.027784,-0.091617,0.091189,-0.051035
1,000001,2000-02,-0.011333,0.000061,-0.011394,0.121265,-0.036489,-0.128943,-0.001473,-0.017608
2,000001,2000-03,0.002729,0.000061,0.002668,0.053877,-0.015575,-0.117326,-0.022687,-0.034834
3,000001,2000-04,0.037017,0.000061,0.036956,0.011737,-0.026647,-0.025242,-0.004163,-0.010473
4,000001,2000-05,-0.055118,0.000061,-0.055179,0.027313,-0.006662,-0.035731,-0.002193,-0.014917


In [175]:
def regress(data):
    X = data[['mkt_risk_premium', 'SMB', 'HML', 'RMW', 'CMA']]
    X = sm.add_constant(X)
    X.replace([np.inf, -np.inf], np.nan, inplace=True)
    X.fillna(0, inplace=True)
    y = data['excess_return']
    model = sm.OLS(np.array(y.astype(float)), X.astype(float))
    result = model.fit()
    params_and_tvalues = np.append(result.params, result.tvalues)
    return params_and_tvalues


betas3 = (tmp_return.groupby('Stkcd').apply(regress, include_groups=False)).apply(
    pd.Series)  # calculate the correlation between the stock return and the risk premium
betas3.columns = ['const', 'risk_premium', 'smb', 'hml', 'rmw', 'cma', 't-const', 't-risk_premium', 't-smb', 't-hml',
                  't-rmw', 't-cma']
betas3.head(5)

,const,risk_premium,smb,hml,rmw,cma,t-const,t-risk_premium,t-smb,t-hml,t-rmw,t-cma
Stkcd,,,,,,,,,,,,
000001,0.003858,1.390265,-0.240255,0.423217,0.226295,0.542351,0.807871,10.598192,-1.875240,3.779933,1.680255,4.786221
000002,0.013893,1.432050,-0.374715,0.658554,0.294480,0.727718,1.987588,7.495638,-2.003525,4.059227,1.497999,4.426882
000003,-0.050966,1.175472,2.002600,-2.025677,0.413053,3.750863,-1.914715,0.576923,1.683281,-1.115726,0.200671,1.832829
000004,0.006712,0.661867,0.699790,-0.554816,-0.637209,-0.586368,0.858424,3.095533,3.350316,-3.045729,-2.900436,-3.183323
000005,-0.007054,0.881924,-0.183349,-0.509954,0.207732,-0.547884,-0.561630,2.581364,-0.540690,-1.722023,0.582785,-1.825550


we will do a mix-OLS regression to get the factor exposure.

In [176]:
X = tmp_return.dropna()[['mkt_risk_premium', 'SMB', 'HML', 'RMW', 'CMA']]
X = sm.add_constant(X)
Y = tmp_return.dropna()['excess_return']
model = sm.OLS(Y.astype(float), X.astype(float))
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          excess_return   R-squared:                       0.240
Model:                            OLS   Adj. R-squared:                  0.240
Method:                 Least Squares   F-statistic:                 4.544e+04
Date:                Sat, 22 Jun 2024   Prob (F-statistic):               0.00
Time:                        11:07:41   Log-Likelihood:             3.8963e+05
No. Observations:              720411   AIC:                        -7.792e+05
Df Residuals:                  720405   BIC:                        -7.792e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.0076      0.000     41.942      0.000       0.007       0.008
mkt_risk_premium     0.9413      0.004    224.010      0.000       0.933       0.950
SMB                  0.3213      0.005     69.176      0.000       0.312       0.330
HML                 -0.2170      0.004    -59.905      0.000      -0.224      -0.210
RMW                 -0.2764      0.005    -56.900      0.000      -0.286      -0.267
CMA                 -0.2353      0.004    -64.218      0.000      -0.242      -0.228
==============================================================================
Omnibus:                  1503100.205   Durbin-Watson:                   2.064
Prob(Omnibus):                  0.000   Jarque-Bera (JB):      62204798484.086
Skew:                          16.952   Prob(JB):                         0.00
Kurtosis:                    1442.153   Cond. No.                         40.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [177]:
with open(r'output/part1/1.4.3 mix_ols_summary1.html', 'w') as tar_file:
    tar_file.write(result.summary().as_html())
with open(r'output/part1/1.4.3 mix_ols_summary2.html', 'w') as tar_file:
    tar_file.write(result.summary2().as_html())

## GRS test

In [178]:
monthly_stock_return.head(10)

,Stkcd,month,monthly_stock_return,market_value,total_assets,total_shareholders_equity,BM ratio,ROE,assets_increasing_rate,rf,monthly_stock_excess_return,S or B,H or L,R or W,C or A,tag
0,000001,2000-06,0.007222,2.813499e+10,4.973234e+10,3.078513e+09,0.109419,0.000000,NaN,0.001832,0.005390,B,L,W,A,BLWA
1,000001,2000-07,0.020960,2.872469e+10,4.973234e+10,3.078513e+09,0.107173,0.000000,NaN,0.001893,0.019067,B,L,W,A,BLWA
2,000001,2000-08,-0.041059,2.754529e+10,4.973234e+10,3.078513e+09,0.111762,0.000000,NaN,0.001893,-0.042952,B,L,W,A,BLWA
3,000001,2000-09,-0.044507,2.631933e+10,4.973234e+10,3.078513e+09,0.116968,0.000000,NaN,0.001832,-0.046339,B,L,W,A,BLWA
4,000001,2000-10,0.034788,2.723492e+10,4.973234e+10,3.078513e+09,0.113036,0.000000,NaN,0.001893,0.032895,B,L,W,A,BLWA
5,000001,2000-11,0.010403,3.014079e+10,4.973234e+10,3.078513e+09,0.102138,0.000000,NaN,0.001832,0.008571,B,L,W,A,BLWA
6,000001,2000-12,-0.062621,2.825334e+10,6.722750e+10,4.738884e+09,0.167728,0.106893,0.351786,0.001893,-0.064514,B,L,R,A,BLRA
7,000001,2001-01,0.031680,2.914842e+10,6.600617e+10,3.517551e+09,0.120677,0.131619,-0.018167,0.001893,0.029787,B,L,R,C,BLRC
8,000001,2001-02,-0.059413,2.741663e+10,6.600617e+10,3.517551e+09,0.128300,0.131619,-0.018167,0.001709,-0.061122,B,L,R,C,BLRC
9,000001,2001-03,0.151171,3.156124e+10,6.600617e+10,3.517551e+09,0.111452,0.131619,-0.018167,0.001893,0.149278,B,L,R,C,BLRC


In [179]:
monthly_stock_return = add_rank(monthly_stock_return)
monthly_stock_return.head(10)

,Stkcd,month,monthly_stock_return,market_value,total_assets,total_shareholders_equity,BM ratio,ROE,assets_increasing_rate,rf,...,ordered_OP,size-OP-rank,ordered_INV,size-INV-rank,ordered_BM_for_32,ordered_OP_for_32,size-BM-OP-rank,ordered_INV_for_32,size-BM-INV-rank,size-OP-INV-rank
0,000001,2000-06,0.007222,2.813499e+10,4.973234e+10,3.078513e+09,0.109419,0.000000,NaN,0.001832,...,1,51,5,55,1,1,B11,4,B14,B14
1,000001,2000-07,0.020960,2.872469e+10,4.973234e+10,3.078513e+09,0.107173,0.000000,NaN,0.001893,...,1,51,5,55,1,1,B11,4,B14,B14
2,000001,2000-08,-0.041059,2.754529e+10,4.973234e+10,3.078513e+09,0.111762,0.000000,NaN,0.001893,...,1,51,5,55,1,1,B11,4,B14,B14
3,000001,2000-09,-0.044507,2.631933e+10,4.973234e+10,3.078513e+09,0.116968,0.000000,NaN,0.001832,...,1,51,5,55,1,1,B11,4,B14,B14
4,000001,2000-10,0.034788,2.723492e+10,4.973234e+10,3.078513e+09,0.113036,0.000000,NaN,0.001893,...,1,51,5,55,1,1,B11,4,B14,B14
5,000001,2000-11,0.010403,3.014079e+10,4.973234e+10,3.078513e+09,0.102138,0.000000,NaN,0.001832,...,1,51,5,55,1,1,B11,4,B14,B14
6,000001,2000-12,-0.062621,2.825334e+10,6.722750e+10,4.738884e+09,0.167728,0.106893,0.351786,0.001893,...,5,55,5,55,1,4,B14,4,B14,B44
7,000001,2001-01,0.031680,2.914842e+10,6.600617e+10,3.517551e+09,0.120677,0.131619,-0.018167,0.001893,...,5,55,1,51,1,4,B14,1,B11,B41
8,000001,2001-02,-0.059413,2.741663e+10,6.600617e+10,3.517551e+09,0.128300,0.131619,-0.018167,0.001709,...,5,55,1,51,1,4,B14,1,B11,B41
9,000001,2001-03,0.151171,3.156124e+10,6.600617e+10,3.517551e+09,0.111452,0.131619,-0.018167,0.001893,...,5,55,1,51,1,4,B14,1,B11,B41


In [180]:
# get portfolios' monthly value-weighted return
size_bm_excess_return3 = monthly_stock_return.groupby(['month', 'size-BM-rank'], observed=False).apply(
    lambda x: np.average(x['monthly_stock_excess_return'], weights=x['market_value']),
    include_groups=False).unstack().fillna(0)
size_op_excess_return3 = monthly_stock_return.groupby(['month', 'size-OP-rank'], observed=False).apply(
    lambda x: np.average(x['monthly_stock_excess_return'], weights=x['market_value']),
    include_groups=False).unstack().fillna(0)
size_inv_excess_return3 = monthly_stock_return.groupby(['month', 'size-INV-rank'], observed=False).apply(
    lambda x: np.average(x['monthly_stock_excess_return'], weights=x['market_value']),
    include_groups=False).unstack().fillna(0)
size_bm_op_excess_return3 = monthly_stock_return.groupby(['month', 'size-BM-OP-rank'], observed=False).apply(
    lambda x: np.average(x['monthly_stock_excess_return'], weights=x['market_value']),
    include_groups=False).unstack().fillna(0)
size_bm_inv_excess_return3 = monthly_stock_return.groupby(['month', 'size-BM-INV-rank'], observed=False).apply(
    lambda x: np.average(x['monthly_stock_excess_return'], weights=x['market_value']),
    include_groups=False).unstack().fillna(0)
size_op_inv_excess_return3 = monthly_stock_return.groupby(['month', 'size-OP-INV-rank'], observed=False).apply(
    lambda x: np.average(x['monthly_stock_excess_return'], weights=x['market_value']),
    include_groups=False).unstack().fillna(0)

In [181]:
def regress(data, factors):
    X = factors
    X = sm.add_constant(X)
    X = X.values
    y = data
    model = sm.OLS(np.array(y.astype(float)), X.astype(float))
    result = model.fit()
    alpha = result.params[0]
    resid = result.resid
    return alpha, resid

In [182]:
factors3 = factors3.dropna()

In [183]:
# get grs stat sheet
grs_size_bm3 = get_grs_stat(size_bm_excess_return3[:-1], factors3)
grs_size_op3 = get_grs_stat(size_op_excess_return3[:-1], factors3)
grs_size_inv3 = get_grs_stat(size_inv_excess_return3[:-1], factors3)
grs_size_bm_op3 = get_grs_stat(size_bm_op_excess_return3[:-1], factors3)
grs_size_bm_inv3 = get_grs_stat(size_bm_inv_excess_return3[:-1], factors3)
grs_size_op_inv3 = get_grs_stat(size_op_inv_excess_return3[:-1], factors3)

/tmp/ipykernel_1676/2627536053.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  excess_return['avg'] = excess_return.mean(axis=1)
/tmp/ipykernel_1676/2627536053.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  excess_return.drop(columns=['avg'], inplace=True)
/tmp/ipykernel_1676/2627536053.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

In [184]:
grs_size_bm3.to_excel(r'output/part1/grs_size_bm3.xlsx', index=False)
grs_size_op3.to_excel(r'output/part1/grs_size_op3.xlsx', index=False)
grs_size_inv3.to_excel(r'output/part1/grs_size_inv3.xlsx', index=False)
grs_size_bm_op3.to_excel(r'output/part1/grs_size_bm_op3.xlsx', index=False)
grs_size_bm_inv3.to_excel(r'output/part1/grs_size_bm_inv3.xlsx', index=False)
grs_size_op_inv3.to_excel(r'output/part1/grs_size_op_inv3.xlsx', index=False)

In [185]:
grs_size_bm_op3

,GRS,p-value of GRS,A|a|,A|a|/A|re|,A(a^2)/A(re^2)
HML,10.977862,1.110223e-16,0.004974,0.842924,0.710521
HML RMW,8.381021,1.110223e-16,0.005708,0.967379,0.935822
HML CMA,10.475146,1.110223e-16,0.005553,0.941127,0.885721
RMW CMA,10.090779,1.110223e-16,0.006736,1.141605,1.303262
HML RMW CMA,7.968054,1.110223e-16,0.005732,0.971453,0.943720


## 1.4.4 correlation of factors from different method

In [186]:
total_factors = pd.concat([factors, factors2, factors3], axis=1)
total_factors.drop(columns=['month'], inplace=True)
total_factors

,mkt_risk_premium,SMB,HML,RMW,CMA,mkt_risk_premium,SMB,HML,RMW,CMA,mkt_risk_premium,SMB,HML,RMW,CMA
0,0.159452,-0.044353,0.163333,-0.058482,0.000000,0.159452,-0.044353,0.163333,-0.058482,0.000000,0.159452,-0.027784,-0.091617,0.091189,-0.051035
1,0.121265,-0.003745,0.161796,-0.097873,0.000000,0.121265,-0.003745,0.161796,-0.097873,0.000000,0.121265,-0.036489,-0.128943,-0.001473,-0.017608
2,0.053877,0.098895,0.043078,0.055793,0.000000,0.053877,0.098895,0.043078,0.055793,0.000000,0.053877,-0.015575,-0.117326,-0.022687,-0.034834
3,0.011737,-0.004885,0.028669,0.000100,0.000000,0.011737,-0.004885,0.028669,0.000100,0.000000,0.011737,-0.026647,-0.025242,-0.004163,-0.010473
4,0.027313,-0.009599,0.008690,-0.097100,0.000000,0.027313,-0.009599,0.008690,-0.097100,0.000000,0.027313,-0.006662,-0.035731,-0.002193,-0.014917
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,-0.057029,-0.009476,-0.010538,0.097489,0.001106,-0.057029,-0.009476,-0.010538,0.097489,0.001106,-0.057029,0.012928,0.015579,0.022890,0.019300
284,-0.011713,-0.023759,0.030768,0.136920,-0.014956,-0.011713,-0.023759,0.030768,0.136920,-0.014956,-0.011713,0.010564,-0.011511,-0.008204,-0.000672
285,-0.022304,0.045157,0.054367,-0.135778,0.018922,-0.022304,0.045157,0.054367,-0.135778,0.018922,-0.022304,0.027305,-0.030953,-0.011858,-0.016493
286,0.002600,0.021006,0.020296,-0.033289,0.018034,0.002600,0.021006,0.020296,-0.033289,0.018034,0.002600,0.017847,-0.021882,-0.000809,-0.004207


In [187]:
total_factors.corr().to_excel(r'output/part1/total_factors_corr.xlsx')
# total_factors.var()